# IF3070 Foundations of Artificial Intelligence | Tugas Besar 2

This notebook serves as a template for the assignment. Please create a copy of this notebook to complete your work. You can add more code blocks, markdown blocks, or new sections if needed.


Group Number: 13

Group Members:
- Jonathan Wiguna (18222019)
- Naomi Pricilla Agustine(18222065)
- Harry Truman Suhalim (18222081)
- Micky Valentino(18222093)

## Import Libraries

In [299]:
import pandas as pd
import numpy as np
import pickle
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import classification_report
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.utils.validation import check_is_fitted, check_array
from imblearn.combine import SMOTETomek
from datetime import datetime
import re

## Import Dataset

In [300]:
train_df = pd.read_csv('https://drive.google.com/uc?id=1629Y3A-tpqB9Oi69heIUk6zPX3K4e1I5')

In [301]:
test_df = pd.read_csv('https://gist.githubusercontent.com/RunningPie/62a453218bd691c00d695f4d1fa05f38/raw/b9214382e6007dd6e267d7693e400e36c0fe4d77/test.csv')

In [302]:
train_df.describe()

,id,URLLength,DomainLength,IsDomainIP,CharContinuationRate,TLDLegitimateProb,URLCharProb,TLDLength,NoOfSubDomain,HasObfuscation,...,Pay,Crypto,HasCopyrightInfo,NoOfImage,NoOfCSS,NoOfJS,NoOfSelfRef,NoOfEmptyRef,NoOfExternalRef,label
count,140404.000000,79765.000000,94085.000000,98274.000000,92362.000000,87531.000000,88333.000000,92673.000000,96344.000000,74684.000000,...,97230.000000,90207.000000,73059.000000,89932.000000,73270.000000,79603.000000,92272.000000,97718.000000,71025.000000,140404.000000
mean,117682.632746,27.701473,19.619387,0.000488,0.918007,0.277482,0.059286,2.729015,1.161442,0.000402,...,0.345706,0.034454,0.751543,41.647489,10.268623,16.461729,104.875900,3.733846,78.104273,0.924831
std,68122.005080,23.140676,5.833808,0.022095,0.159142,0.248115,0.008063,0.533310,0.445054,0.020038,...,0.475600,0.182393,0.432121,102.538702,133.321659,21.001274,219.137035,24.192458,187.106965,0.263664
min,1.000000,14.000000,4.000000,0.000000,0.000000,0.000000,0.001229,2.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,58689.500000,23.000000,16.000000,0.000000,0.913043,0.012927,0.055542,2.000000,1.000000,0.000000,...,0.000000,0.000000,1.000000,10.000000,2.000000,6.000000,24.000000,0.000000,13.000000,1.000000
50%,117421.500000,26.000000,19.000000,0.000000,1.000000,0.522907,0.060264,3.000000,1.000000,0.000000,...,0.000000,0.000000,1.000000,23.000000,5.000000,12.000000,69.000000,0.000000,39.000000,1.000000
75%,176724.250000,30.000000,23.000000,0.000000,1.000000,0.522907,0.064391,3.000000,1.000000,0.000000,...,1.000000,0.000000,1.000000,45.000000,12.000000,22.000000,132.000000,3.000000,99.000000,1.000000
max,235795.000000,4054.000000,93.000000,1.000000,1.000000,0.522907,0.088766,13.000000,7.000000,1.000000,...,1.000000,1.000000,1.000000,8956.000000,35820.000000,2828.000000,26596.000000,4887.000000,27516.000000,1.000000


In [303]:
test_df.describe()

,id,URLLength,DomainLength,IsDomainIP,CharContinuationRate,TLDLegitimateProb,URLCharProb,TLDLength,NoOfSubDomain,HasObfuscation,...,Bank,Pay,Crypto,HasCopyrightInfo,NoOfImage,NoOfCSS,NoOfJS,NoOfSelfRef,NoOfEmptyRef,NoOfExternalRef
count,10000.000000,7574.000000,7937.000000,7160.000000,6055.000000,6311.000000,6499.000000,7025.000000,6321.000000,6602.000000,...,6418.000000,6183.000000,7234.000000,6494.000000,6379.000000,6751.000000,6045.000000,7286.000000,7651.000000,7888.000000
mean,116685.950300,35.459863,21.803578,0.002654,0.831552,0.257059,0.055020,2.773096,1.168644,0.002424,...,0.122001,0.215106,0.022809,0.421928,23.065057,5.443194,9.094458,57.170738,2.194223,46.400355
std,67807.841787,30.508312,9.600899,0.051449,0.222677,0.251699,0.010885,0.591598,0.636971,0.049173,...,0.327312,0.410929,0.149304,0.493905,65.569730,10.555097,14.534499,119.740073,15.384745,243.280431
min,48.000000,13.000000,4.000000,0.000000,0.146341,0.000000,0.001223,2.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,58401.750000,24.000000,16.000000,0.000000,0.666667,0.005977,0.049361,2.000000,1.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,116405.000000,28.000000,20.000000,0.000000,1.000000,0.079963,0.057273,3.000000,1.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,4.000000,1.000000,3.000000,2.000000,0.000000,4.000000
75%,175534.500000,35.000000,25.000000,0.000000,1.000000,0.522907,0.062408,3.000000,1.000000,0.000000,...,0.000000,0.000000,0.000000,1.000000,26.000000,6.000000,13.000000,75.000000,1.000000,45.000000
max,235770.000000,779.000000,91.000000,1.000000,1.000000,0.522907,0.082057,8.000000,7.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1836.000000,177.000000,284.000000,2436.000000,724.000000,14117.000000


In [304]:
train_df = train_df.drop(columns=['FILENAME'])
test_df = test_df.drop(columns=['FILENAME'])


In [305]:
categorical = ['TLD',  'Title', 'Domain','URL',  'IsDomainIP', 'HasObfuscation', 'IsHTTPS', 'HasTitle', 'HasFavicon', 'Robots', 'IsResponsive', 'HasDescription', 'HasExternalFormSubmit', 'HasSocialNet', 'HasSubmitButton', 'HasHiddenFields', 'HasPasswordField', 'Bank', 'Pay', 'Crypto', 'HasCopyrightInfo']

non_categorical = [col for col in train_df.columns if col not in categorical]

# 1. Split Training Set and Validation Set

Splitting the training and validation set works as an early diagnostic towards the performance of the model we train. This is done before the preprocessing steps to **avoid data leakage inbetween the sets**. If you want to use k-fold cross-validation, split the data later and do the cleaning and preprocessing separately for each split.

Note: For training, you should use the data contained in the `train` folder given by the TA. The `test` data is only used for kaggle submission.

In [306]:
train_set, val_set = train_test_split(train_df, test_size=0.3,stratify=train_df['label'], random_state=42)

In [307]:
train_set

,id,URL,URLLength,Domain,DomainLength,IsDomainIP,TLD,CharContinuationRate,TLDLegitimateProb,URLCharProb,...,Pay,Crypto,HasCopyrightInfo,NoOfImage,NoOfCSS,NoOfJS,NoOfSelfRef,NoOfEmptyRef,NoOfExternalRef,label
88211,147927,https://www.autogespot.com,NaN,NaN,18.0,NaN,NaN,1.000000,0.522907,0.066821,...,0.0,0.0,1.0,NaN,1.0,5.0,120.0,NaN,NaN,1
84658,141904,https://www.filmandtvnow.com,27.0,NaN,20.0,NaN,com,1.000000,NaN,0.054564,...,0.0,0.0,1.0,136.0,NaN,NaN,341.0,23.0,365.0,1
110399,185372,https://outlookcom249.yolasite.com/,34.0,outlookcom249.yolasite.com,NaN,0.0,com,0.636364,NaN,0.059770,...,NaN,0.0,0.0,0.0,NaN,0.0,0.0,0.0,NaN,0
43324,72399,https://arubahost.it.sell.20-57-10-180.cprapid...,57.0,NaN,42.0,0.0,com,0.342105,0.522907,0.045228,...,0.0,0.0,NaN,0.0,0.0,NaN,0.0,NaN,0.0,0
51618,86331,https://www.britishhair.com,NaN,NaN,19.0,0.0,com,1.000000,0.522907,0.057839,...,1.0,0.0,1.0,NaN,4.0,NaN,NaN,4.0,12.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63704,106495,https://www.daydreamingmaven.com,NaN,NaN,24.0,0.0,NaN,1.000000,0.522907,0.059770,...,0.0,0.0,1.0,18.0,NaN,17.0,110.0,NaN,NaN,1
1267,2124,https://www.alicegal.com,NaN,NaN,NaN,0.0,com,NaN,0.522907,NaN,...,NaN,0.0,NaN,13.0,NaN,NaN,43.0,0.0,NaN,1
81354,136276,https://www.orionzjewels.com,NaN,NaN,20.0,0.0,NaN,1.000000,0.522907,NaN,...,1.0,0.0,NaN,2075.0,NaN,24.0,205.0,12.0,NaN,1
46464,77719,https://www.identitagolose.com,29.0,www.identitagolose.com,NaN,0.0,com,NaN,0.522907,NaN,...,NaN,0.0,NaN,NaN,12.0,NaN,75.0,NaN,101.0,1


In [308]:
val_set

,id,URL,URLLength,Domain,DomainLength,IsDomainIP,TLD,CharContinuationRate,TLDLegitimateProb,URLCharProb,...,Pay,Crypto,HasCopyrightInfo,NoOfImage,NoOfCSS,NoOfJS,NoOfSelfRef,NoOfEmptyRef,NoOfExternalRef,label
125796,211196,NaN,19.0,www.sofia.bg,12.0,0.0,NaN,1.000000,0.000395,0.056725,...,0.0,NaN,1.0,41.0,NaN,NaN,NaN,2.0,NaN,1
16799,27992,NaN,NaN,www.valeriebiel.com,19.0,0.0,NaN,1.000000,0.522907,0.064115,...,0.0,0.0,NaN,15.0,NaN,9.0,23.0,0.0,18.0,1
105765,177510,https://www.salvatorebattaglia.com.au,36.0,www.salvatorebattaglia.com.au,29.0,NaN,NaN,0.863636,0.010086,NaN,...,0.0,0.0,1.0,NaN,NaN,23.0,NaN,1.0,NaN,1
90405,151597,https://www.alturashomes.com,NaN,NaN,20.0,0.0,com,1.000000,0.522907,0.065633,...,1.0,0.0,1.0,7.0,NaN,11.0,34.0,NaN,NaN,1
90059,151044,https://www.merricksart.com,NaN,www.merricksart.com,19.0,0.0,NaN,NaN,0.522907,0.065927,...,0.0,0.0,NaN,191.0,NaN,41.0,207.0,NaN,208.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71388,119484,https://www.gih.se,17.0,NaN,10.0,0.0,se,1.000000,0.004019,NaN,...,0.0,NaN,1.0,NaN,10.0,18.0,98.0,NaN,NaN,1
134957,226588,NaN,23.0,www.ensenada.net,16.0,0.0,NaN,NaN,NaN,0.069015,...,0.0,0.0,0.0,NaN,8.0,12.0,12.0,0.0,11.0,1
105353,176806,https://www.behoneybee.com,NaN,www.behoneybee.com,18.0,0.0,com,1.000000,0.522907,0.064695,...,1.0,NaN,NaN,30.0,NaN,NaN,123.0,NaN,17.0,1
105638,177289,https://www.mbda.gov,19.0,www.mbda.gov,12.0,NaN,NaN,NaN,0.003328,0.054471,...,NaN,NaN,NaN,NaN,2.0,8.0,59.0,1.0,40.0,1


# 2. Data Cleaning and Preprocessing

This step is the first thing to be done once a Data Scientist have grasped a general knowledge of the data. Raw data is **seldom ready for training**, therefore steps need to be taken to clean and format the data for the Machine Learning model to interpret.

By performing data cleaning and preprocessing, you ensure that your dataset is ready for model training, leading to more accurate and reliable machine learning results. These steps are essential for transforming raw data into a format that machine learning algorithms can effectively learn from and make predictions.

We will give some common methods for you to try, but you only have to **at least implement one method for each process**. For each step that you will do, **please explain the reason why did you do that process. Write it in a markdown cell under the code cell you wrote.**

## A. Data Cleaning

**Data cleaning** is the crucial first step in preparing your dataset for machine learning. Raw data collected from various sources is often messy and may contain errors, missing values, and inconsistencies. Data cleaning involves the following steps:

1. **Handling Missing Data:** Identify and address missing values in the dataset. This can include imputing missing values, removing rows or columns with excessive missing data, or using more advanced techniques like interpolation.

2. **Dealing with Outliers:** Identify and handle outliers, which are data points significantly different from the rest of the dataset. Outliers can be removed or transformed to improve model performance.

3. **Data Validation:** Check for data integrity and consistency. Ensure that data types are correct, categorical variables have consistent labels, and numerical values fall within expected ranges.

4. **Removing Duplicates:** Identify and remove duplicate rows, as they can skew the model's training process and evaluation metrics.

5. **Feature Engineering**: Create new features or modify existing ones to extract relevant information. This step can involve scaling, normalizing, or encoding features for better model interpretability.

### I. Handling Missing Data

Missing data can adversely affect the performance and accuracy of machine learning models. There are several strategies to handle missing data in machine learning:

1. **Data Imputation:**

    a. **Mean, Median, or Mode Imputation:** For numerical features, you can replace missing values with the mean, median, or mode of the non-missing values in the same feature. This method is simple and often effective when data is missing at random.

    b. **Constant Value Imputation:** You can replace missing values with a predefined constant value (e.g., 0) if it makes sense for your dataset and problem.

    c. **Imputation Using Predictive Models:** More advanced techniques involve using predictive models to estimate missing values. For example, you can train a regression model to predict missing numerical values or a classification model to predict missing categorical values.

2. **Deletion of Missing Data:**

    a. **Listwise Deletion:** In cases where the amount of missing data is relatively small, you can simply remove rows with missing values from your dataset. However, this approach can lead to a loss of valuable information.

    b. **Column (Feature) Deletion:** If a feature has a large number of missing values and is not critical for your analysis, you can consider removing that feature altogether.

3. **Domain-Specific Strategies:**

    a. **Domain Knowledge:** In some cases, domain knowledge can guide the imputation process. For example, if you know that missing values are related to a specific condition, you can impute them accordingly.

4. **Imputation Libraries:**

    a. **Scikit-Learn:** Scikit-Learn provides a `SimpleImputer` class that can handle basic imputation strategies like mean, median, and mode imputation.

    b. **Fancyimpute:** Fancyimpute is a Python library that offers more advanced imputation techniques, including matrix factorization, k-nearest neighbors, and deep learning-based methods.

The choice of imputation method should be guided by the nature of your data, the amount of missing data, the problem you are trying to solve, and the assumptions you are willing to make.

Identify Missing Values

In [309]:
# Identifikasi missing values
print("Jumlah missing values:")
print(train_set.isnull().sum())
print()

# Menghitung persentase missing values
missing_percent = (train_set.isnull().sum() / len(train_set)) * 100
print("Persentase missing values:")
print(missing_percent)

Jumlah missing values:
id                                0
URL                           30409
URLLength                     42367
Domain                        49000
DomainLength                  32385
IsDomainIP                    29433
TLD                           31859
CharContinuationRate          33573
TLDLegitimateProb             36984
URLCharProb                   36397
TLDLength                     33509
NoOfSubDomain                 30654
HasObfuscation                45938
NoOfObfuscatedChar            46752
ObfuscationRatio              45111
NoOfLettersInURL              44345
LetterRatioInURL              46049
NoOfDegitsInURL               41234
DegitRatioInURL               37518
NoOfEqualsInURL               43122
NoOfQMarkInURL                30843
NoOfAmpersandInURL            31749
NoOfOtherSpecialCharsInURL    33217
SpacialCharRatioInURL         43878
IsHTTPS                       34569
LineOfCode                    48310
LargestLineLength             47596
HasTi

In [310]:
missing_percent = (train_set.isnull().sum() / len(train_set)) * 100
print("Persentase missing values:")
print(missing_percent)

Persentase missing values:
id                             0.000000
URL                           30.940559
URLLength                     43.107588
Domain                        49.856535
DomainLength                  32.951100
IsDomainIP                    29.947498
TLD                           32.415905
CharContinuationRate          34.159867
TLDLegitimateProb             37.630492
URLCharProb                   37.033231
TLDLength                     34.094748
NoOfSubDomain                 31.189841
HasObfuscation                46.741011
NoOfObfuscatedChar            47.569240
ObfuscationRatio              45.899554
NoOfLettersInURL              45.120164
LetterRatioInURL              46.853951
NoOfDegitsInURL               41.954783
DegitRatioInURL               38.173826
NoOfEqualsInURL               43.875786
NoOfQMarkInURL                31.382145
NoOfAmpersandInURL            32.303982
NoOfOtherSpecialCharsInURL    33.797644
SpacialCharRatioInURL         44.645001
IsHTTPS      

Data Imputation

In [311]:
# Assign url dengan modus ketika tidak ada Domain dan IsHTTPS
if 'URL' in train_set.columns:
    # Mencari url paling sering/modus
    most_frequent_url = train_set['URL'].mode()[0]

    mask_both_missing = pd.isna(train_set['IsHTTPS']) & pd.isna(train_set['Domain'])
    train_set.loc[mask_both_missing & pd.isna(train_set['URL']), 'URL'] = most_frequent_url

    mask_one_exists = ((pd.notna(train_set['IsHTTPS']) & pd.isna(train_set['Domain'])) | 
            (pd.isna(train_set['IsHTTPS']) & pd.notna(train_set['Domain'])))
    train_set.loc[mask_one_exists & pd.isna(train_set['URL']), 'URL'] = most_frequent_url

# Assign Domain dan TLD jika terdapat URL
for index, row in train_set.iterrows():
    if pd.notna(row['URL']):
        try:
            # Cek HTTPS
            train_set.at[index, 'IsHTTPS'] = 1 if row['URL'].startswith('https://') else 0
            
            parsed = urlparse(row['URL'])
            if not parsed.netloc: # Buat domain yang tidak punya protokol
                parsed = urlparse("http://" + row['URL'])
            
            domain = parsed.netloc or parsed.path

            # Extract TLD
            if '.' in domain:
                tld = domain.split('.')[-1]
                
                # Update Domain and TLD 
                if pd.isna(train_set.at[index, 'Domain']):
                    train_set.at[index, 'Domain'] = domain
                if pd.isna(train_set.at[index, 'TLD']):
                    train_set.at[index, 'TLD'] = tld
        except:
            continue
    # Jika URL tidak ada tapi domaion ada
    elif pd.notna(row['Domain']) and pd.isna(row['TLD']):
        domain = row['Domain']
        if '.' in domain:
            train_set.at[index, 'TLD'] = domain.split('.')[-1]
    
    # Isi URL jika Domain dan IsHTTPS ada
    if pd.isna(row['URL']) and pd.notna(row['Domain']) and pd.notna(row['IsHTTPS']):
        protocol = 'https://' if row['IsHTTPS'] == 1 else 'http://'
        train_set.at[index, 'URL'] = f"{protocol}www.{row['Domain']}"

# Proses URL-based features
if 'URL' in train_set.columns:
    # Update URLLength berdasarkan URL yang ada
    mask = train_set['URL'].notna()
    train_set.loc[mask, 'URLLength'] = train_set.loc[mask, 'URL'].str.len()
    
    # Update letter dan digit counts berdasarkan URL yang ada
    train_set.loc[mask, 'NoOfLettersInURL'] = train_set.loc[mask, 'URL'].str.count('[a-zA-Z]')
    train_set.loc[mask, 'NoOfDegitsInURL'] = train_set.loc[mask, 'URL'].str.count('[0-9]')
    train_set.loc[mask, 'NoOfEqualsInURL'] = train_set.loc[mask, 'URL'].str.count('=')
    train_set.loc[mask, 'NoOfQMarkInURL'] = train_set.loc[mask, 'URL'].str.count(r'\?')
    train_set.loc[mask, 'NoOfAmpersandInURL'] = train_set.loc[mask, 'URL'].str.count('&')
    
    url_lengths = train_set.loc[mask, 'URL'].str.len()
    train_set.loc[mask, 'LetterRatioInURL'] = train_set.loc[mask, 'NoOfLettersInURL'] / url_lengths
    train_set.loc[mask, 'DegitRatioInURL'] = train_set.loc[mask, 'NoOfDegitsInURL'] / url_lengths
    
    special_chars_pattern = '[^a-zA-Z0-9=?&]'
    train_set.loc[mask, 'NoOfOtherSpecialCharsInURL'] = train_set.loc[mask, 'URL'].str.count(special_chars_pattern)
    train_set.loc[mask, 'SpacialCharRatioInURL'] = train_set.loc[mask, 'NoOfOtherSpecialCharsInURL'] / url_lengths
    
    # CharContinuationRate
    char_series = train_set.loc[mask, 'URL']
    continuation_rates = []
    
    for url in char_series:
        if len(url) < 2:
            continuation_rates.append(0)
        else:
            continuous = sum(1 for i in range(len(url)-1) if url[i] == url[i+1])
            continuation_rates.append(continuous / (len(url)-1))
    
    train_set.loc[mask, 'CharContinuationRate'] = continuation_rates

# Proses Domain-based features
if 'Domain' in train_set.columns:
    # Update DomainLength berdasarkan Domain yang ada
    mask = train_set['Domain'].notna()
    train_set.loc[mask, 'DomainLength'] = train_set.loc[mask, 'Domain'].str.len()
    
    # Update IsDomainIP berdasarkan Domain yang ada
    train_set.loc[mask, 'IsDomainIP'] = train_set.loc[mask, 'Domain'].apply(
        lambda x: float(bool(re.match(r'^(?:[0-9]{1,3}\.){3}[0-9]{1,3}$', str(x)))))
    
    # Update NoOfSubDomain berdasarkan Domain yang ada
    train_set.loc[mask, 'NoOfSubDomain'] = train_set.loc[mask, 'Domain'].str.count(r'\.')

# Proses TLD-based features
if 'TLD' in train_set.columns:
    mask = train_set['TLD'].notna()
    train_set.loc[mask, 'TLDLength'] = train_set.loc[mask, 'TLD'].str.len()

# Hubungan HasObfuscation dengan NoOfObfuscatedChar
mask = train_set['NoOfObfuscatedChar'] == 0
train_set.loc[mask & train_set['HasObfuscation'].isna(), 'HasObfuscation'] = 0

mask = train_set['NoOfObfuscatedChar'] > 0
train_set.loc[mask & train_set['HasObfuscation'].isna(), 'HasObfuscation'] = 1

# NoOfSelfRedirect tidak boleh lebih besar dari NoOfURLRedirect
mask = train_set['NoOfURLRedirect'].notna() & train_set['NoOfSelfRedirect'].notna()
invalid_mask = mask & (train_set['NoOfSelfRedirect'] > train_set['NoOfURLRedirect'])
train_set.loc[invalid_mask, 'NoOfSelfRedirect'] = train_set.loc[invalid_mask, 'NoOfURLRedirect']

# HasExternalFormSubmit berdasarkan NoOfExternalRef
mask = train_set['NoOfExternalRef'] > 0
train_set.loc[mask & train_set['HasExternalFormSubmit'].isna(), 'HasExternalFormSubmit'] = 1

mask = train_set['NoOfExternalRef'] == 0
train_set.loc[mask & train_set['HasExternalFormSubmit'].isna(), 'HasExternalFormSubmit'] = 0

# Proses title 
mask = train_set['Title'].notna()
train_set.loc[mask & train_set['HasTitle'].isna(), 'HasTitle'] = 1
train_set.loc[~mask & train_set['HasTitle'].isna(), 'HasTitle'] = 0

# Isi missing values yang tersisa
boolean_cols = ['HasObfuscation', 'IsHTTPS', 'HasTitle',
                'HasFavicon', 'IsResponsive', 'HasDescription',
                'HasExternalFormSubmit', 'HasSocialNet', 'HasSubmitButton',
                'HasHiddenFields', 'HasPasswordField', 'HasCopyrightInfo']

count_cols = ['NoOfObfuscatedChar', 'NoOfURLRedirect', 'NoOfSelfRedirect', 'NoOfPopup',
              'NoOfiFrame', 'NoOfImage', 'NoOfCSS', 'NoOfJS',
              'NoOfSelfRef', 'NoOfEmptyRef', 'NoOfExternalRef']

# Handle missing values sisanya
for col in boolean_cols:
    if col in train_set.columns and train_set[col].isnull().any():
        train_set[col] = train_set[col].fillna(train_set[col].mode()[0])

for col in count_cols:
    if col in train_set.columns and train_set[col].isnull().any():
        train_set[col] = train_set[col].fillna(train_set[col].median())

# Handle remaining numerical columns
numerical_cols = train_set.select_dtypes(include=['float64', 'int64']).columns
remaining_cols = [col for col in numerical_cols if col not in boolean_cols + count_cols]
for col in remaining_cols:
    if train_set[col].isnull().any():
        train_set[col] = train_set[col].fillna(train_set[col].median())

print("\nJumlah missing values setelah diisi:")
print(train_set.isnull().sum())


Jumlah missing values setelah diisi:
id                                0
URL                               0
URLLength                         0
Domain                        49000
DomainLength                      0
IsDomainIP                        0
TLD                           28628
CharContinuationRate              0
TLDLegitimateProb                 0
URLCharProb                       0
TLDLength                         0
NoOfSubDomain                     0
HasObfuscation                    0
NoOfObfuscatedChar                0
ObfuscationRatio                  0
NoOfLettersInURL                  0
LetterRatioInURL                  0
NoOfDegitsInURL                   0
DegitRatioInURL                   0
NoOfEqualsInURL                   0
NoOfQMarkInURL                    0
NoOfAmpersandInURL                0
NoOfOtherSpecialCharsInURL        0
SpacialCharRatioInURL             0
IsHTTPS                           0
LineOfCode                        0
LargestLineLength         

In [312]:
# Assign url dengan modus ketika tidak ada Domain dan IsHTTPS
if 'URL' in val_set.columns:
    # Mencari url paling sering/modus
    most_frequent_url = val_set['URL'].mode()[0]

    mask_both_missing = pd.isna(val_set['IsHTTPS']) & pd.isna(val_set['Domain'])
    val_set.loc[mask_both_missing & pd.isna(val_set['URL']), 'URL'] = most_frequent_url

    mask_one_exists = ((pd.notna(val_set['IsHTTPS']) & pd.isna(val_set['Domain'])) | 
            (pd.isna(val_set['IsHTTPS']) & pd.notna(val_set['Domain'])))
    val_set.loc[mask_one_exists & pd.isna(val_set['URL']), 'URL'] = most_frequent_url

# Assign Domain dan TLD jika terdapat URL
for index, row in val_set.iterrows():
    if pd.notna(row['URL']):
        try:
            # Cek HTTPS
            val_set.at[index, 'IsHTTPS'] = 1 if row['URL'].startswith('https://') else 0
            
            parsed = urlparse(row['URL'])
            if not parsed.netloc: # Buat domain yang tidak punya protokol
                parsed = urlparse("http://" + row['URL'])
            
            domain = parsed.netloc or parsed.path

            # Extract TLD
            if '.' in domain:
                tld = domain.split('.')[-1]
                
                # Update Domain and TLD 
                if pd.isna(val_set.at[index, 'Domain']):
                    val_set.at[index, 'Domain'] = domain
                if pd.isna(val_set.at[index, 'TLD']):
                    val_set.at[index, 'TLD'] = tld
        except:
            continue
    # Jika URL tidak ada tapi domaion ada
    elif pd.notna(row['Domain']) and pd.isna(row['TLD']):
        domain = row['Domain']
        if '.' in domain:
            val_set.at[index, 'TLD'] = domain.split('.')[-1]
    
    # Isi URL jika Domain dan IsHTTPS ada
    if pd.isna(row['URL']) and pd.notna(row['Domain']) and pd.notna(row['IsHTTPS']):
        protocol = 'https://' if row['IsHTTPS'] == 1 else 'http://'
        val_set.at[index, 'URL'] = f"{protocol}www.{row['Domain']}"

# Proses URL-based features
if 'URL' in val_set.columns:
    # Update URLLength berdasarkan URL yang ada
    mask = val_set['URL'].notna()
    val_set.loc[mask, 'URLLength'] = val_set.loc[mask, 'URL'].str.len()
    
    # Update letter dan digit counts berdasarkan URL yang ada
    val_set.loc[mask, 'NoOfLettersInURL'] = val_set.loc[mask, 'URL'].str.count('[a-zA-Z]')
    val_set.loc[mask, 'NoOfDegitsInURL'] = val_set.loc[mask, 'URL'].str.count('[0-9]')
    val_set.loc[mask, 'NoOfEqualsInURL'] = val_set.loc[mask, 'URL'].str.count('=')
    val_set.loc[mask, 'NoOfQMarkInURL'] = val_set.loc[mask, 'URL'].str.count(r'\?')
    val_set.loc[mask, 'NoOfAmpersandInURL'] = val_set.loc[mask, 'URL'].str.count('&')
    
    url_lengths = val_set.loc[mask, 'URL'].str.len()
    val_set.loc[mask, 'LetterRatioInURL'] = val_set.loc[mask, 'NoOfLettersInURL'] / url_lengths
    val_set.loc[mask, 'DegitRatioInURL'] = val_set.loc[mask, 'NoOfDegitsInURL'] / url_lengths
    
    special_chars_pattern = '[^a-zA-Z0-9=?&]'
    val_set.loc[mask, 'NoOfOtherSpecialCharsInURL'] = val_set.loc[mask, 'URL'].str.count(special_chars_pattern)
    val_set.loc[mask, 'SpacialCharRatioInURL'] = val_set.loc[mask, 'NoOfOtherSpecialCharsInURL'] / url_lengths
    
    # CharContinuationRate
    char_series = val_set.loc[mask, 'URL']
    continuation_rates = []
    
    for url in char_series:
        if len(url) < 2:
            continuation_rates.append(0)
        else:
            continuous = sum(1 for i in range(len(url)-1) if url[i] == url[i+1])
            continuation_rates.append(continuous / (len(url)-1))
    
    val_set.loc[mask, 'CharContinuationRate'] = continuation_rates

# Proses Domain-based features
if 'Domain' in val_set.columns:
    # Update DomainLength berdasarkan Domain yang ada
    mask = val_set['Domain'].notna()
    val_set.loc[mask, 'DomainLength'] = val_set.loc[mask, 'Domain'].str.len()
    
    # Update IsDomainIP berdasarkan Domain yang ada
    val_set.loc[mask, 'IsDomainIP'] = val_set.loc[mask, 'Domain'].apply(
        lambda x: float(bool(re.match(r'^(?:[0-9]{1,3}\.){3}[0-9]{1,3}$', str(x)))))
    
    # Update NoOfSubDomain berdasarkan Domain yang ada
    val_set.loc[mask, 'NoOfSubDomain'] = val_set.loc[mask, 'Domain'].str.count(r'\.')

# Proses TLD-based features
if 'TLD' in val_set.columns:
    mask = val_set['TLD'].notna()
    val_set.loc[mask, 'TLDLength'] = val_set.loc[mask, 'TLD'].str.len()

# Hubungan HasObfuscation dengan NoOfObfuscatedChar
mask = val_set['NoOfObfuscatedChar'] == 0
val_set.loc[mask & val_set['HasObfuscation'].isna(), 'HasObfuscation'] = 0

mask = val_set['NoOfObfuscatedChar'] > 0
val_set.loc[mask & val_set['HasObfuscation'].isna(), 'HasObfuscation'] = 1

# NoOfSelfRedirect tidak boleh lebih besar dari NoOfURLRedirect
mask = val_set['NoOfURLRedirect'].notna() & val_set['NoOfSelfRedirect'].notna()
invalid_mask = mask & (val_set['NoOfSelfRedirect'] > val_set['NoOfURLRedirect'])
val_set.loc[invalid_mask, 'NoOfSelfRedirect'] = val_set.loc[invalid_mask, 'NoOfURLRedirect']

# HasExternalFormSubmit berdasarkan NoOfExternalRef
mask = val_set['NoOfExternalRef'] > 0
val_set.loc[mask & val_set['HasExternalFormSubmit'].isna(), 'HasExternalFormSubmit'] = 1

mask = val_set['NoOfExternalRef'] == 0
val_set.loc[mask & val_set['HasExternalFormSubmit'].isna(), 'HasExternalFormSubmit'] = 0

# Proses title 
mask = val_set['Title'].notna()
val_set.loc[mask & val_set['HasTitle'].isna(), 'HasTitle'] = 1
val_set.loc[~mask & val_set['HasTitle'].isna(), 'HasTitle'] = 0

# Isi missing values yang tersisa
boolean_cols = ['HasObfuscation', 'IsHTTPS', 'HasTitle',
                'HasFavicon', 'IsResponsive', 'HasDescription',
                'HasExternalFormSubmit', 'HasSocialNet', 'HasSubmitButton',
                'HasHiddenFields', 'HasPasswordField', 'HasCopyrightInfo']

count_cols = ['NoOfObfuscatedChar', 'NoOfURLRedirect', 'NoOfSelfRedirect', 'NoOfPopup',
              'NoOfiFrame', 'NoOfImage', 'NoOfCSS', 'NoOfJS',
              'NoOfSelfRef', 'NoOfEmptyRef', 'NoOfExternalRef']

# Handle missing values sisanya
for col in boolean_cols:
    if col in val_set.columns and val_set[col].isnull().any():
        val_set[col] = val_set[col].fillna(val_set[col].mode()[0])

for col in count_cols:
    if col in val_set.columns and val_set[col].isnull().any():
        val_set[col] = val_set[col].fillna(val_set[col].median())

# Handle remaining numerical columns
numerical_cols = val_set.select_dtypes(include=['float64', 'int64']).columns
remaining_cols = [col for col in numerical_cols if col not in boolean_cols + count_cols]
for col in remaining_cols:
    if val_set[col].isnull().any():
        val_set[col] = val_set[col].fillna(val_set[col].median())

print("\nJumlah missing values setelah diisi:")
print(val_set.isnull().sum())


Jumlah missing values setelah diisi:
id                                0
URL                               0
URLLength                         0
Domain                        21197
DomainLength                      0
IsDomainIP                        0
TLD                           12179
CharContinuationRate              0
TLDLegitimateProb                 0
URLCharProb                       0
TLDLength                         0
NoOfSubDomain                     0
HasObfuscation                    0
NoOfObfuscatedChar                0
ObfuscationRatio                  0
NoOfLettersInURL                  0
LetterRatioInURL                  0
NoOfDegitsInURL                   0
DegitRatioInURL                   0
NoOfEqualsInURL                   0
NoOfQMarkInURL                    0
NoOfAmpersandInURL                0
NoOfOtherSpecialCharsInURL        0
SpacialCharRatioInURL             0
IsHTTPS                           0
LineOfCode                        0
LargestLineLength         

### II. Dealing with Outliers

Outliers are data points that significantly differ from the majority of the data. They can be unusually high or low values that do not fit the pattern of the rest of the dataset. Outliers can significantly impact model performance, so it is important to handle them properly.

Some methods to handle outliers:
1. **Imputation**: Replace with mean, median, or a boundary value.
2. **Clipping**: Cap values to upper and lower limits.
3. **Transformation**: Use log, square root, or power transformations to reduce their influence.
4. **Model-Based**: Use algorithms robust to outliers (e.g., tree-based models, Huber regression).

In [313]:
# import pandas as pd
# import numpy as np

# # Misalkan val_set adalah DataFrame Anda dan non_categorical adalah daftar kolom non-kategorikal
# threshold_percentage = 0.03  # Batas maksimal 3% outlier

# outlier = True
# while outlier:
#     change = False
#     for col in non_categorical:
#         if val_set[col].skew() > 1 or val_set[col].skew() < -1:
#             # Menggunakan IQR untuk mendeteksi outlier
#             Q1 = val_set[col].quantile(0.25)
#             Q3 = val_set[col].quantile(0.75)
#             IQR = Q3 - Q1
#             # Mendapatkan indeks outlier
#             outlier_indices = val_set[(val_set[col] < Q1 - 1.5 * IQR) | (val_set[col] > Q3 + 1.5 * IQR)].index
#         else:
#             # Menggunakan z-score untuk mendeteksi outlier
#             z_scores = np.abs((val_set[col] - val_set[col].mean()) / val_set[col].std())
#             outlier_indices = val_set[z_scores > 3].index
        
#         # Hitung persentase outlier
#         outlier_percentage = len(outlier_indices) / len(val_set)
#         if 0 < outlier_percentage < threshold_percentage:
#             val_set = val_set.drop(outlier_indices)
#             change = True
#         elif outlier_percentage > threshold_percentage:
#             Q1 = val_set[col].quantile(0.25)
#             Q3 = val_set[col].quantile(0.75)
#             IQR = Q3 - Q1
#             lower_bound = Q1 - 1.5 * IQR
#             upper_bound = Q3 + 1.5 * IQR

#             val_set[col] = np.where(val_set[col] < lower_bound, lower_bound, val_set[col])
#             val_set[col] = np.where(val_set[col] > upper_bound, upper_bound, val_set[col])

#     print (change)
#     if not(change):
#         outlier = False

#     # Reset index setelah penghapusan outlier
#     val_set.reset_index(drop=True, inplace=True)


In [314]:
# import pandas as pd
# import numpy as np

# # Misalkan train_set adalah DataFrame Anda dan non_categorical adalah daftar kolom non-kategorikal
# threshold_percentage = 0.03  # Batas maksimal 3% outlier
# outlier = True
# while outlier:
#     change = False
#     for col in non_categorical:
#         if train_set[col].skew() > 1 or train_set[col].skew() < -1:
#             # Menggunakan IQR untuk mendeteksi outlier
#             Q1 = train_set[col].quantile(0.25)
#             Q3 = train_set[col].quantile(0.75)
#             IQR = Q3 - Q1
#             # Mendapatkan indeks outlier
#             outlier_indices = train_set[(train_set[col] < Q1 - 1.5 * IQR) | (train_set[col] > Q3 + 1.5 * IQR)].index
#         else:
#             # Menggunakan z-score untuk mendeteksi outlier
#             z_scores = np.abs((train_set[col] - train_set[col].mean()) / train_set[col].std())
#             outlier_indices = train_set[z_scores > 3].index
        
#         # Hitung persentase outlier
#         outlier_percentage = len(outlier_indices) / len(train_set)
#         if 0 < outlier_percentage < threshold_percentage:
#             train_set = train_set.drop(outlier_indices)
#             change = True
#         elif outlier_percentage > threshold_percentage:
#             Q1 = train_set[col].quantile(0.25)
#             Q3 = train_set[col].quantile(0.75)
#             IQR = Q3 - Q1
#             lower_bound = Q1 - 1.5 * IQR
#             upper_bound = Q3 + 1.5 * IQR

#             train_set[col] = np.where(train_set[col] < lower_bound, lower_bound, train_set[col])
#             train_set[col] = np.where(train_set[col] > upper_bound, upper_bound, train_set[col])

#     print (change)
#     if not(change):
#         outlier = False

#     # Reset index setelah penghapusan outlier
#     train_set.reset_index(drop=True, inplace=True)


In [315]:
# for col in non_categorical:
#     if train_set[col].skew() > 1 or train_set[col].skew() < -1:
#         # Menggunakan IQR untuk mendeteksi outlier
#         Q1 = train_set[col].quantile(0.25)
#         Q3 = train_set[col].quantile(0.75)
#         IQR = Q3 - Q1
#         # Mendapatkan indeks outlier
#         outlier_indices = train_set[(train_set[col] < Q1 - 1.5 * IQR) | (train_set[col] > Q3 + 1.5 * IQR)].index
#     else:
#         # Menggunakan z-score untuk mendeteksi outlier
#         z_scores = np.abs((train_set[col] - train_set[col].mean()) / train_set[col].std())
#         outlier_indices = train_set[z_scores > 3].index
    
#     # Hitung persentase outlier
#     outlier_percentage = len(outlier_indices) / len(train_set) * 100

#     print (outlier_percentage)

In [316]:
# for col in non_categorical:
#     if val_set[col].skew() > 1 or val_set[col].skew() < -1:
#         # Menggunakan IQR untuk mendeteksi outlier
#         Q1 = val_set[col].quantile(0.25)
#         Q3 = val_set[col].quantile(0.75)
#         IQR = Q3 - Q1
#         # Mendapatkan indeks outlier
#         outlier_indices = val_set[(val_set[col] < Q1 - 1.5 * IQR) | (val_set[col] > Q3 + 1.5 * IQR)].index
#     else:
#         # Menggunakan z-score untuk mendeteksi outlier
#         z_scores = np.abs((val_set[col] - val_set[col].mean()) / val_set[col].std())
#         outlier_indices = val_set[z_scores > 3].index
    
#     # Hitung persentase outlier
#     outlier_percentage = len(outlier_indices) / len(val_set) * 100

#     print (outlier_percentage)

In [317]:
import numpy as np

def handle_outliers(df, non_categorical, label_value, threshold_percentage=0.03):
    subset = df[df['label'] == label_value].copy()
    outlier = True
    while outlier:
        change = False
        for col in non_categorical:
            if subset[col].skew() > 1 or subset[col].skew() < -1:
                # iqr buat skewed
                Q1 = subset[col].quantile(0.25)
                Q3 = subset[col].quantile(0.75)
                IQR = Q3 - Q1
                outlier_indices = subset[(subset[col] < Q1 - 1.5 * IQR) | (subset[col] > Q3 + 1.5 * IQR)].index
            else:
                # z-score buat less skewed
                z_scores = np.abs((subset[col] - subset[col].mean()) / subset[col].std())
                outlier_indices = subset[z_scores > 3].index
            
            # percentage outlier
            outlier_percentage = len(outlier_indices) / len(subset)
            if 0 < outlier_percentage < threshold_percentage:
                subset = subset.drop(outlier_indices)
                change = True
            elif outlier_percentage > threshold_percentage:
                Q1 = subset[col].quantile(0.25)
                Q3 = subset[col].quantile(0.75)
                IQR = Q3 - Q1
                lower_bound = Q1 - 1.5 * IQR
                upper_bound = Q3 + 1.5 * IQR

                # cap outliers
                subset[col] = np.where(subset[col] < lower_bound, lower_bound, subset[col])
                subset[col] = np.where(subset[col] > upper_bound, upper_bound, subset[col])
        
        print(f"change for label {label_value}: {change}")
        if not change:
            outlier = False
        
        # reset indexing
        subset.reset_index(drop=True, inplace=True)
    
    return subset


Dipisah-pisah dengan tujuan si yang label 0 ga kehapus karena dianggap outlier

In [318]:
# outlier di training set buat label=1
train_label_1 = handle_outliers(train_set, non_categorical, label_value=1)

# outlier di training set buat label=0
train_label_0 = handle_outliers(train_set, non_categorical, label_value=0)

# combine
train_set_cleaned = pd.concat([train_label_1, train_label_0], ignore_index=True)

# outlier di validation set buat label=1
val_label_1 = handle_outliers(val_set, non_categorical, label_value=1)

# outlier di validation set buat label=0
val_label_0 = handle_outliers(val_set, non_categorical, label_value=0)

# combine
val_set_cleaned = pd.concat([val_label_1, val_label_0], ignore_index=True)

# cek distribution
print("\ntraining set label outlier handling:")
print(train_set_cleaned['label'].value_counts())

print("\nval set label distribution outlier handling:")
print(val_set_cleaned['label'].value_counts())

train_set = train_set_cleaned
val_set = val_set_cleaned


change for label 1: True
change for label 1: True
change for label 1: True
change for label 1: False
change for label 0: True
change for label 0: True
change for label 0: True
change for label 0: True
change for label 0: True
change for label 0: True
change for label 0: True
change for label 0: True
change for label 0: False
change for label 1: True
change for label 1: True
change for label 1: False
change for label 0: True
change for label 0: True
change for label 0: True
change for label 0: True
change for label 0: False

training set label outlier handling:
label
1    87305
0     5960
Name: count, dtype: int64

val set label distribution outlier handling:
label
1    36305
0     2801
Name: count, dtype: int64


In [319]:
train_set['label'].value_counts()

label
1    87305
0     5960
Name: count, dtype: int64

### III. Remove Duplicates
Handling duplicate values is crucial because they can compromise data integrity, leading to inaccurate analysis and insights. Duplicate entries can bias machine learning models, causing overfitting and reducing their ability to generalize to new data. They also inflate the dataset size unnecessarily, increasing computational costs and processing times. Additionally, duplicates can distort statistical measures and lead to inconsistencies, ultimately affecting the reliability of data-driven decisions and reporting. Ensuring data quality by removing duplicates is essential for accurate, efficient, and consistent analysis.

In [320]:
jumlah_duplikat = train_set.duplicated().sum()
print(f"Jumlah baris duplikat: {jumlah_duplikat}")

Jumlah baris duplikat: 0


In [321]:
jumlah_duplikat = val_set.duplicated().sum()
print(f"Jumlah baris duplikat: {jumlah_duplikat}")

Jumlah baris duplikat: 0


In [322]:
# Write your code here
train_set = train_set.drop_duplicates()

In [323]:
# Write your code here
val_set = val_set.drop_duplicates()

In [324]:
jumlah_duplikat = train_set.duplicated().sum()
print(f"Jumlah baris duplikat: {jumlah_duplikat}")

Jumlah baris duplikat: 0


In [325]:
jumlah_duplikat = val_set.duplicated().sum()
print(f"Jumlah baris duplikat: {jumlah_duplikat}")

Jumlah baris duplikat: 0


### IV. Feature Engineering

**Feature engineering** involves creating new features (input variables) or transforming existing ones to improve the performance of machine learning models. Feature engineering aims to enhance the model's ability to learn patterns and make accurate predictions from the data. It's often said that "good features make good models."

1. **Feature Selection:** Feature engineering can involve selecting the most relevant and informative features from the dataset. Removing irrelevant or redundant features not only simplifies the model but also reduces the risk of overfitting.

2. **Creating New Features:** Sometimes, the existing features may not capture the underlying patterns effectively. In such cases, engineers create new features that provide additional information. For example:
   
   - **Polynomial Features:** Engineers may create new features by taking the square, cube, or other higher-order terms of existing numerical features. This can help capture nonlinear relationships.
   
   - **Interaction Features:** Interaction features are created by combining two or more existing features. For example, if you have features "length" and "width," you can create an "area" feature by multiplying them.

3. **Binning or Discretization:** Continuous numerical features can be divided into bins or categories. For instance, age values can be grouped into bins like "child," "adult," and "senior."

4. **Domain-Specific Feature Engineering:** Depending on the domain and problem, engineers may create domain-specific features. For example, in fraud detection, features related to transaction history and user behavior may be engineered to identify anomalies.

Feature engineering is both a creative and iterative process. It requires a deep understanding of the data, domain knowledge, and experimentation to determine which features will enhance the model's predictive power.

In [326]:
print(len(train_df.columns))

55


In [327]:
# # Buat fitur baru dari karakteristik URL
# if 'URL' in train_set.columns:
#    # Hitung karakter spesial
#    train_set['special_char_count'] = train_set['URL'].str.count(r'[^a-zA-Z0-9]').fillna(0)
#    # Hitung digit
#    train_set['digit_count'] = train_set['URL'].str.count(r'[0-9]').fillna(0)

# # Buat fitur interaksi
# if 'URLLength' in train_set.columns and 'DomainLength' in train_set.columns:
#    train_set['url_domain_ratio'] = train_set['URLLength'] / train_set['DomainLength'].replace(0, 1)

# # Buat fitur binary dari TLD
# if 'TLD' in train_set.columns:
#    common_tlds = ['com', 'org', 'net', 'edu']
#    train_set['is_common_tld'] = train_set['TLD'].isin(common_tlds).astype(int)

# # Buat aggregate features
# security_cols = ['HasObfuscation', 'IsHTTPS', 'HasHiddenFields']
# train_set['security_score'] = train_set[security_cols].sum(axis=1)

# # Cek fitur baru
# print("\nJumlah kolom setelah feature engineering:", len(train_set.columns))
# print("\nBeberapa baris dengan fitur baru:")
# print(train_set[['special_char_count', 'digit_count', 'url_domain_ratio',
#           'is_common_tld', 'security_score']].head())

In [328]:
# Buat fitur baru dari karakteristik URL
if 'URL' in val_set.columns:
   # Hitung karakter spesial
   val_set['special_char_count'] = val_set['URL'].str.count(r'[^a-zA-Z0-9]').fillna(0)
   # Hitung digit
   val_set['digit_count'] = val_set['URL'].str.count(r'[0-9]').fillna(0)

# Buat fitur interaksi
if 'URLLength' in val_set.columns and 'DomainLength' in val_set.columns:
   val_set['url_domain_ratio'] = val_set['URLLength'] / val_set['DomainLength'].replace(0, 1)

# Buat fitur binary dari TLD
if 'TLD' in val_set.columns:
   common_tlds = ['com', 'org', 'net', 'edu']
   val_set['is_common_tld'] = val_set['TLD'].isin(common_tlds).astype(int)

# Buat aggregate features
security_cols = ['HasObfuscation', 'IsHTTPS', 'HasHiddenFields']
val_set['security_score'] = val_set[security_cols].sum(axis=1)

# Cek fitur baru
print("\nJumlah kolom setelah feature engineering:", len(val_set.columns))
print("\nBeberapa baris dengan fitur baru:")
print(val_set[['special_char_count', 'digit_count', 'url_domain_ratio',
          'is_common_tld', 'security_score']].head())


Jumlah kolom setelah feature engineering: 60

Beberapa baris dengan fitur baru:
   special_char_count  digit_count  url_domain_ratio  is_common_tld  \
0                   6            0          2.000000              0   
1                   6            0          1.631579              1   
2                   6            0          1.275862              0   
3                   5            0          1.400000              1   
4                   5            0          1.421053              0   

   security_score  
0             2.0  
1             2.0  
2             2.0  
3             2.0  
4             2.0  


In [329]:
# sample_size = 30000
# train_sample = train_set.sample(n=sample_size, random_state=42)

# numerical_features = train_sample.select_dtypes(include=['float64', 'int64']).columns
# X_numeric = train_sample[numerical_features].drop('label', axis=1)
# y = train_sample['label']

# correlation_matrix = X_numeric.corr()

# threshold = 0.8
# high_corr_features = []
# for i in range(len(correlation_matrix.columns)):
#     for j in range(i):
#         if abs(correlation_matrix.iloc[i, j]) > threshold:
#             col_i = correlation_matrix.columns[i]
#             col_j = correlation_matrix.columns[j]
#             high_corr_features.append((col_i, col_j, correlation_matrix.iloc[i, j]))

# print("\nhigh correlation pairs (> 0.8):")
# for f1, f2, corr in high_corr_features:
#     print(f"{f1} - {f2}: {corr:.4f}")
    
# #scale features
# scaler = StandardScaler()
# X_scaled = scaler.fit_transform(X_filtered)
# X_scaled = pd.DataFrame(X_scaled, columns=X_filtered.columns)

# # std --> ga guna remove aja
# non_constant_features = X_numeric.columns[X_numeric.std() != 0].tolist()
# X_filtered = X_numeric[non_constant_features]

# #count mi
# mi_scores = mutual_info_classif(X_scaled, y)
# mi_df = pd.DataFrame({
#     'Feature': X_filtered.columns,
#     'MI_Score': mi_scores
# })
# mi_df = mi_df.sort_values('MI_Score', ascending=False)

# print("\nimportant features:")
# print(mi_df.head(10).to_string())

In [330]:
# Define the features to drop with reasons
features_to_drop = {
    #korelasi tinggi
    'NoOfLettersInURL': 'almost perfect correlation sama URLLength (0.998)',
    'NoOfOtherSpecialCharsInURL': 'strong correlation URLLength (0.876)',
    'DomainLength': 'strong correlation URLLength (0.845)',
    
    # varianceny 0, ga contribute ke modeling
    'NoOfObfuscatedChar': 'konstan, std 0',
    'URLTitleMatchScore': 'konstan, std 0',
    'NoOfURLRedirect': 'konstan, std 0',
    'ObfuscationRatio': 'konstan, std 0',
    'DomainTitleMatchScore': 'konstan, std 0',
    'DegitRatioInURL': 'konstan, std 0',
    'NoOfEqualsInURL': 'konstan, std 0',
    'HasObfuscation': 'konstan, std 0',
    'NoOfSelfRedirect': 'konstan, std 0',
    'NoOfPopup': 'konstan, std 0',
    'NoOfQMarkInURL': 'konstan, std 0',
    'NoOfAmpersandInURL': 'konstan, std 0',
    'NoOfDegitsInURL': 'konstan, std 0',
    'NoOfSubDomain': 'konstan, std 0',
    
    # uda kegambarin di feature lain/raw text
    'Domain': 'raw text',
    'URL': 'raw text',
    'Title': 'raw text',
    'LetterRatioInURL': 'derived',
    'SpacialCharRatioInURL': 'derived',
    
    # missing too much, low mi
    'NoOfCSS': 'High missing values, low importance',
    'HasCopyrightInfo': 'High missing values, low importance',
    'HasPasswordField': 'High missing values, low importance',
    'HasSocialNet': 'High missing values, low importance',
    'NoOfExternalRef': 'High missing, info captured in HasExternalFormSubmit'
}

#drop
train_set = train_set.drop(columns=list(features_to_drop.keys()), errors='ignore')
val_set = val_set.drop(columns=list(features_to_drop.keys()), errors='ignore')

#same columns
common_columns = train_set.columns.intersection(val_set.columns)
train_set = train_set[common_columns]
val_set = val_set[common_columns]

#verify
print("\nnumber of remaining features:", len(train_set.columns))
print("training set:", train_set.columns.tolist())
print("validation set:", val_set.columns.tolist())


# shapess
print("\nNew shapes:")
print(f"training set: {train_set.shape}")
print(f"validation set: {val_set.shape}")



number of remaining features: 28
training set: ['id', 'URLLength', 'IsDomainIP', 'TLD', 'CharContinuationRate', 'TLDLegitimateProb', 'URLCharProb', 'TLDLength', 'IsHTTPS', 'LineOfCode', 'LargestLineLength', 'HasTitle', 'HasFavicon', 'Robots', 'IsResponsive', 'HasDescription', 'NoOfiFrame', 'HasExternalFormSubmit', 'HasSubmitButton', 'HasHiddenFields', 'Bank', 'Pay', 'Crypto', 'NoOfImage', 'NoOfJS', 'NoOfSelfRef', 'NoOfEmptyRef', 'label']
validation set: ['id', 'URLLength', 'IsDomainIP', 'TLD', 'CharContinuationRate', 'TLDLegitimateProb', 'URLCharProb', 'TLDLength', 'IsHTTPS', 'LineOfCode', 'LargestLineLength', 'HasTitle', 'HasFavicon', 'Robots', 'IsResponsive', 'HasDescription', 'NoOfiFrame', 'HasExternalFormSubmit', 'HasSubmitButton', 'HasHiddenFields', 'Bank', 'Pay', 'Crypto', 'NoOfImage', 'NoOfJS', 'NoOfSelfRef', 'NoOfEmptyRef', 'label']

New shapes:
training set: (93265, 28)
validation set: (39106, 28)


In [331]:
def engineer_features(dataset):
    features = dataset.copy()
    
    
    # ui content - ngambil juga dari tucil
    ui_features = ['HasFavicon',  'HasSubmitButton', 'HasTitle', 'HasDescription', 'IsResponsive']
    features['UIQualityScore'] = features[ui_features].sum(axis=1)
    
    # terkait duit n pembayaran
    financial_features = ['Bank', 'Pay', 'Crypto']
    features['FinancialRiskScore'] = (
        features[financial_features].sum(axis=1) * 
        (1 - features['IsHTTPS'])  #kalo ga https worse
    )
    
    # domain
    features['DomainTrustScore'] = (
        features['TLDLegitimateProb'] * 
        (1 + features['IsHTTPS']) *  #kalo https kasi plus
        (1 - features['IsDomainIP'])  #kalo domainnya ip = veri sus kurangin
    )
    
    # kompleksitas code
    features['ContentComplexity'] = (
        np.log1p(features['LineOfCode']) * 
        np.log1p(features['LargestLineLength'])
    )
    
    # ref pattern
    features['RefPatternScore'] = (
        features['NoOfSelfRef'] / 
        (features['NoOfSelfRef'] + features['NoOfEmptyRef']).clip(lower=1)
    )
    
    return features

train_set_engineered = engineer_features(train_set)
val_set_engineered = engineer_features(val_set)

train_set = train_set_engineered
val_set = val_set_engineered

new_features = [
    'UIQualityScore',
    'FinancialRiskScore',
    'DomainTrustScore',
    'ContentComplexity',
    'RefPatternScore'
]

print("\ncorrelation with label:")
correlations = train_set_engineered[new_features + ['label']].corr()['label'].drop('label')
print(correlations.sort_values(ascending=False))

print("\nnew features stats:")
print(train_set_engineered[new_features].describe())

# recheck label distribution
print("label information:")
print("Type:", train_set['label'].dtype)
print("\nvalue counts:")
print(train_set['label'].value_counts())



correlation with label:
ContentComplexity     0.708072
RefPatternScore       0.493757
UIQualityScore        0.414023
DomainTrustScore      0.112665
FinancialRiskScore    0.044471
Name: label, dtype: float64

new features stats:
       UIQualityScore  FinancialRiskScore  DomainTrustScore  \
count    93265.000000        93265.000000      93265.000000   
mean         4.055101            0.078154          0.656448   
std          0.971644            0.313280          0.442152   
min          0.000000            0.000000          0.000000   
25%          4.000000            0.000000          0.159926   
50%          4.000000            0.000000          1.045814   
75%          5.000000            0.000000          1.045814   
max          5.000000            3.000000          1.045814   

       ContentComplexity  RefPatternScore  
count       93265.000000     93265.000000  
mean           53.491172         0.924458  
std             7.710822         0.235264  
min             3.536297   

In [332]:
selected_features = [
    'RefPatternScore',
    'UIQualityScore',
    'ContentComplexity'
]

print("\ncorrelations with label:")
correlations = train_set_engineered[selected_features + ['label']].corr()['label'].drop('label')
print(correlations.sort_values(ascending=False))

#drop yang korelasi jelek
train_set = train_set.drop(columns = ['DomainTrustScore', 'FinancialRiskScore'])
val_set = val_set.drop(columns = ['DomainTrustScore', 'FinancialRiskScore'])



correlations with label:
ContentComplexity    0.708072
RefPatternScore      0.493757
UIQualityScore       0.414023
Name: label, dtype: float64


## B. Data Preprocessing

**Data preprocessing** is a broader step that encompasses both data cleaning and additional transformations to make the data suitable for machine learning algorithms. Its primary goals are:

1. **Feature Scaling:** Ensure that numerical features have similar scales. Common techniques include Min-Max scaling (scaling to a specific range) or standardization (mean-centered, unit variance).

2. **Encoding Categorical Variables:** Machine learning models typically work with numerical data, so categorical variables need to be encoded. This can be done using one-hot encoding, label encoding, or more advanced methods like target encoding.

3. **Handling Imbalanced Classes:** If dealing with imbalanced classes in a binary classification task, apply techniques such as oversampling, undersampling, or using different evaluation metrics to address class imbalance.

4. **Dimensionality Reduction:** Reduce the number of features using techniques like Principal Component Analysis (PCA) or feature selection to simplify the model and potentially improve its performance.

5. **Normalization:** Normalize data to achieve a standard distribution. This is particularly important for algorithms that assume normally distributed data.

Drop Column that Useless

In [333]:
# train_set = train_set.drop(columns=['Title', 'Domain', 'URL'])

In [334]:
# val_set = val_set.drop(columns=['Title', 'Domain', 'URL'])

### Notes on Preprocessing processes

It is advised to create functions or classes that have the same/similar type of inputs and outputs, so you can add, remove, or swap the order of the processes easily. You can implement the functions or classes by yourself

or

use `sklearn` library. To create a new preprocessing component in `sklearn`, implement a corresponding class that includes:
1. Inheritance to `BaseEstimator` and `TransformerMixin`
2. The method `fit`
3. The method `transform`

In [335]:
# Example

# from sklearn.base import BaseEstimator, TransformerMixin

# class FeatureEncoder(BaseEstimator, TransformerMixin):

#     def fit(self, X, y=None):

#         # Fit the encoder here

#         return self

#     def transform(self, X):
#         X_encoded = X.copy()

#         # Encode the categorical variables here

#         return X_encoded

### I. Feature Scaling

**Feature scaling** is a preprocessing technique used in machine learning to standardize the range of independent variables or features of data. The primary goal of feature scaling is to ensure that all features contribute equally to the training process and that machine learning algorithms can work effectively with the data.

Here are the main reasons why feature scaling is important:

1. **Algorithm Sensitivity:** Many machine learning algorithms are sensitive to the scale of input features. If the scales of features are significantly different, some algorithms may perform poorly or take much longer to converge.

2. **Distance-Based Algorithms:** Algorithms that rely on distances or similarities between data points, such as k-nearest neighbors (KNN) and support vector machines (SVM), can be influenced by feature scales. Features with larger scales may dominate the distance calculations.

3. **Regularization:** Regularization techniques, like L1 (Lasso) and L2 (Ridge) regularization, add penalty terms based on feature coefficients. Scaling ensures that all features are treated equally in the regularization process.

Common methods for feature scaling include:

1. **Min-Max Scaling (Normalization):** This method scales features to a specific range, typically [0, 1]. It's done using the following formula:

   $$X' = \frac{X - X_{min}}{X_{max} - X_{min}}$$

   - Here, $X$ is the original feature value, $X_{min}$ is the minimum value of the feature, and $X_{max}$ is the maximum value of the feature.  
<br />
<br />
2. **Standardization (Z-score Scaling):** This method scales features to have a mean (average) of 0 and a standard deviation of 1. It's done using the following formula:

   $$X' = \frac{X - \mu}{\sigma}$$

   - $X$ is the original feature value, $\mu$ is the mean of the feature, and $\sigma$ is the standard deviation of the feature.  
<br />
<br />
3. **Robust Scaling:** Robust scaling is a method that scales features to the interquartile range (IQR) and is less affected by outliers. It's calculated as:

   $$X' = \frac{X - Q1}{Q3 - Q1}$$

   - $X$ is the original feature value, $Q1$ is the first quartile (25th percentile), and $Q3$ is the third quartile (75th percentile) of the feature.  
<br />
<br />
4. **Log Transformation:** In cases where data is highly skewed or has a heavy-tailed distribution, taking the logarithm of the feature values can help stabilize the variance and improve scaling.

The choice of scaling method depends on the characteristics of your data and the requirements of your machine learning algorithm. **Min-max scaling and standardization are the most commonly used techniques and work well for many datasets.**

Scaling should be applied separately to each training and test set to prevent data leakage from the test set into the training set. Additionally, **some algorithms may not require feature scaling, particularly tree-based models.**

In [336]:
class NumericStandardScaler(BaseEstimator, TransformerMixin):
    def __init__(self, exclude_label='label', label_column='label'):
        self.exclude_label = exclude_label
        self.label_column = label_column
        self.scaler = StandardScaler()

    def fit(self, X, y=None):
        # Pilih hanya kolom numerik
        X_numeric = X.select_dtypes(include=['float64', 'int64'])

        # Exclude kolom label jika ada
        if self.exclude_label and self.label_column in X_numeric.columns:
            X_numeric = X_numeric.drop(columns=[self.label_column])

        # Validasi data numerik
        X_numeric = check_array(X_numeric, dtype=np.float64)

        # Simpan nama kolom numerik selain kolom label
        self.numeric_cols = [
            col for col in X.select_dtypes(include=['float64', 'int64']).columns
            if col != self.label_column
        ]

        # Fit scaler pada data numerik
        self.scaler.fit(X_numeric)
        return self

    def transform(self, X):
        X_transformed = X.copy()

        # Transform hanya kolom numerik selain label
        X_transformed[self.numeric_cols] = self.scaler.transform(
            X_transformed[self.numeric_cols]
        )
        return X_transformed

In [337]:
class NumericMinMaxScaler(BaseEstimator, TransformerMixin):
      def __init__(self, exclude_label=True, label_column='label', exclude_id=True, id_column='id'):
          self.exclude_label = exclude_label
          self.label_column = label_column
          self.exclude_id = exclude_id
          self.id_column = id_column
          self.scaler = MinMaxScaler()
  
      def fit(self, X, y=None):
          # Identify numeric columns
          self.numeric_cols_ = X.select_dtypes(include=['float64', 'int64']).columns
          
          # Exclude label column if specified
          if self.exclude_label and self.label_column in self.numeric_cols_:
              self.numeric_cols_ = self.numeric_cols_.drop(self.label_column)
  
          # Exclude id column if specified
          if self.exclude_id and self.id_column in self.numeric_cols_:
              self.numeric_cols_ = self.numeric_cols_.drop(self.id_column)
  
          # Fit the scaler only on numeric columns
          if len(self.numeric_cols_) > 0:
              self.scaler.fit(X[self.numeric_cols_])
              self.is_fitted_ = True
          return self
  
      def transform(self, X):
          # Check if fit has been called
          check_is_fitted(self, 'is_fitted_')
  
          X_transformed = X.copy()
          if len(self.numeric_cols_) > 0:
              scaled_data = self.scaler.transform(X_transformed[self.numeric_cols_])
              X_transformed[self.numeric_cols_] = scaled_data
          return X_transformed

In [338]:
# # Identifikasi kolom numerik dari train_set yang sudah ada
# numeric_cols = train_set.select_dtypes(include=['float64', 'int64']).columns
# numeric_cols = numeric_cols.drop('label') if 'label' in numeric_cols else numeric_cols

# # Pilih hanya kolom numerik dari train_set dan val_set
# train_set_numeric = train_set[numeric_cols]
# val_set_numeric = val_set[numeric_cols]

# # Standardization
# scaler = StandardScaler()
# train_set_scaled = scaler.fit_transform(train_set_numeric)
# val_set_scaled = scaler.transform(val_set_numeric)

# # Convert kembali ke DataFrame
# train_set_scaled = pd.DataFrame(train_set_scaled, columns=train_set_numeric.columns)
# val_set_scaled = pd.DataFrame(val_set_scaled, columns=val_set_numeric.columns)

# # Cek hasil
# train_set_scaled.describe()

### II. Feature Encoding

**Feature encoding**, also known as **categorical encoding**, is the process of converting categorical data (non-numeric data) into a numerical format so that it can be used as input for machine learning algorithms. Most machine learning models require numerical data for training and prediction, so feature encoding is a critical step in data preprocessing.

Categorical data can take various forms, including:

1. **Nominal Data:** Categories with no intrinsic order, like colors or country names.  

2. **Ordinal Data:** Categories with a meaningful order but not necessarily equidistant, like education levels (e.g., "high school," "bachelor's," "master's").

There are several common methods for encoding categorical data:

1. **Label Encoding:**

   - Label encoding assigns a unique integer to each category in a feature.
   - It's suitable for ordinal data where there's a clear order among categories.
   - For example, if you have an "education" feature with values "high school," "bachelor's," and "master's," you can encode them as 0, 1, and 2, respectively.
<br />
<br />
2. **One-Hot Encoding:**

   - One-hot encoding creates a binary (0 or 1) column for each category in a nominal feature.
   - It's suitable for nominal data where there's no inherent order among categories.
   - Each category becomes a new feature, and the presence (1) or absence (0) of a category is indicated for each row.
<br />
<br />
3. **Target Encoding (Mean Encoding):**

   - Target encoding replaces each category with the mean of the target variable for that category.
   - It's often used for classification problems.

In [339]:
class FrequencyEncoder(BaseEstimator, TransformerMixin):
    def __init__(self, columns=None, drop_original=True):
        self.columns = columns
        self.drop_original = drop_original

    def fit(self, X, y=None):
        # If columns not specified, find all object and category columns
        if self.columns is None:
            self.columns_ = X.select_dtypes(include=['object', 'category']).columns.tolist()
        else:
            self.columns_ = self.columns
        
        # Ensure all specified columns exist in the dataframe
        missing_cols = [col for col in self.columns_ if col not in X.columns]
        if missing_cols:
            raise ValueError(f"Columns not found in input data: {missing_cols}")
        
        # Store frequencies in an attribute with trailing underscore
        self.frequencies_ = {}
        for col in self.columns_:
            self.frequencies_[col] = X[col].value_counts(normalize=True)
        
        self.is_fitted_ = True
        return self

    def transform(self, X):
        # Check if fit has been called
        check_is_fitted(self, 'is_fitted_')
        
        X_transformed = X.copy()
        
        # Check if all columns exist in input data
        missing_cols = [col for col in self.columns_ if col not in X_transformed.columns]
        if missing_cols:
            raise ValueError(f"Columns not found in input data during transform: {missing_cols}")
            
        for col in self.columns_:
            # Create new column name for encoded values
            encoded_col_name = f"{col}_encoded"
            
            # Encode the column
            X_transformed[encoded_col_name] = X_transformed[col].map(self.frequencies_[col])
            X_transformed[encoded_col_name] = X_transformed[encoded_col_name].fillna(0)
            
            # Drop original column if specified
            if self.drop_original:
                X_transformed = X_transformed.drop(columns=[col])
        
        return X_transformed

In [340]:
train_set

,id,URLLength,IsDomainIP,TLD,CharContinuationRate,TLDLegitimateProb,URLCharProb,TLDLength,IsHTTPS,LineOfCode,...,Pay,Crypto,NoOfImage,NoOfJS,NoOfSelfRef,NoOfEmptyRef,label,UIQualityScore,ContentComplexity,RefPatternScore
0,141904,28.0,0.0,com,0.148148,0.522907,0.054564,3.0,1.0,1306.5,...,0.0,0.0,54.0,13.0,166.0,5.0,1,4.0,56.648618,0.970760
1,86331,27.0,0.0,com,0.192308,0.522907,0.057839,3.0,1.0,1014.0,...,1.0,0.0,23.0,13.0,69.0,4.0,1,5.0,58.450921,0.945205
2,123732,35.0,0.0,NaN,0.117647,0.032650,0.060261,2.0,1.0,1306.5,...,0.0,0.0,51.0,13.0,69.0,4.0,1,4.0,56.648618,0.945205
3,139283,48.0,0.0,NaN,0.085106,0.522907,0.058785,3.0,0.0,1014.0,...,0.0,0.0,10.0,7.5,25.0,0.0,1,4.0,50.647557,1.000000
4,142686,34.0,0.0,NaN,0.121212,0.079963,0.056044,3.0,1.0,1014.0,...,0.0,0.0,19.0,13.0,59.0,0.0,1,5.0,54.649553,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
93260,3213,35.0,0.0,com,0.147059,0.522907,0.060261,3.0,0.0,2.0,...,0.0,0.0,0.0,1.0,1.0,0.0,0,4.0,8.672795,1.000000
93261,44004,23.0,0.0,org,0.181818,0.079963,0.060261,3.0,0.0,1014.0,...,0.0,0.0,0.0,13.0,0.0,0.0,0,2.0,54.649553,0.000000
93262,181826,48.0,0.0,NaN,0.085106,0.522907,0.047742,3.0,0.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0,2.0,5.705046,0.000000
93263,197670,30.0,0.0,nz,0.137931,0.001993,0.060261,2.0,0.0,3.0,...,0.0,0.0,0.0,13.0,69.0,0.0,0,1.0,10.943849,1.000000


In [341]:
val_set

,id,URLLength,IsDomainIP,TLD,CharContinuationRate,TLDLegitimateProb,URLCharProb,TLDLength,IsHTTPS,LineOfCode,...,Pay,Crypto,NoOfImage,NoOfJS,NoOfSelfRef,NoOfEmptyRef,label,UIQualityScore,ContentComplexity,RefPatternScore
0,211196,24.0,0.0,bg,0.260870,0.000395,0.056725,2.0,1.0,1283.0,...,0.0,0.0,41.0,12.0,69.0,2.0,1,3.0,56.510776,0.971831
1,27992,31.0,0.0,com,0.200000,0.522907,0.064115,3.0,1.0,1013.0,...,0.0,0.0,15.0,9.0,23.0,0.0,1,5.0,52.082441,1.000000
2,177510,37.0,0.0,NaN,0.138889,0.010086,0.060276,2.0,1.0,851.0,...,0.0,0.0,23.0,19.5,69.0,1.0,1,3.0,53.272624,0.985714
3,151597,28.0,0.0,com,0.148148,0.522907,0.065633,3.0,1.0,1166.0,...,1.0,0.0,7.0,11.0,34.0,0.0,1,4.0,54.847924,1.000000
4,151044,27.0,0.0,NaN,0.192308,0.522907,0.065927,3.0,1.0,1013.0,...,0.0,0.0,52.5,19.5,166.0,0.0,1,4.0,58.755104,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39101,148129,20.0,0.0,NaN,0.105263,0.522907,0.068827,3.0,1.0,1013.0,...,0.0,0.0,0.0,12.0,69.0,0.0,0,2.0,54.646931,1.000000
39102,176289,44.0,0.0,co,0.046512,0.005977,0.052905,2.0,1.0,1013.0,...,0.0,0.0,23.0,12.0,69.0,0.0,0,1.0,54.646931,1.000000
39103,108323,32.0,0.0,NaN,0.129032,0.522907,0.060276,4.0,1.0,2.0,...,0.0,0.0,23.0,0.0,0.0,0.0,0,2.0,4.106250,0.000000
39104,148508,48.0,0.0,com,0.127660,0.522907,0.060276,3.0,1.0,1013.0,...,0.0,0.0,23.0,0.0,69.0,0.0,0,3.0,54.646931,1.000000


### III. Handling Imbalanced Dataset

**Handling imbalanced datasets** is important because imbalanced data can lead to several issues that negatively impact the performance and reliability of machine learning models. Here are some key reasons:

1. **Biased Model Performance**:

 - Models trained on imbalanced data tend to be biased towards the majority class, leading to poor performance on the minority class. This can result in misleading accuracy metrics.

2. **Misleading Accuracy**:

 - High overall accuracy can be misleading in imbalanced datasets. For example, if 95% of the data belongs to one class, a model that always predicts the majority class will have 95% accuracy but will fail to identify the minority class.

3. **Poor Generalization**:

 - Models trained on imbalanced data may not generalize well to new, unseen data, especially if the minority class is underrepresented.


Some methods to handle imbalanced datasets:
1. **Resampling Methods**:

 - Oversampling: Increase the number of instances in the minority class by duplicating or generating synthetic samples (e.g., SMOTE).
 - Undersampling: Reduce the number of instances in the majority class to balance the dataset.

2. **Evaluation Metrics**:

 - Use appropriate evaluation metrics such as precision, recall, F1-score, ROC-AUC, and confusion matrix instead of accuracy to better assess model performance on imbalanced data.

3. **Algorithmic Approaches**:

 - Use algorithms that are designed to handle imbalanced data, such as decision trees, random forests, or ensemble methods.
 - Adjust class weights in algorithms to give more importance to the minority class.

In [342]:
print("distribution:")
print(train_df['label'].value_counts())
print("\npercentage distribution:")
print(train_df['label'].value_counts(normalize=True) * 100)

distribution:
label
1    129850
0     10554
Name: count, dtype: int64

percentage distribution:
label
1    92.48312
0     7.51688
Name: proportion, dtype: float64


In [343]:
# columns_to_drop_imbalanced = ['id', 'FILENAME', 'URL', 'Title', 'Domain']
# X_imbalanced = df.drop(columns_to_drop_imbalanced + ['label'], axis=1)
# X_imbalanced = pd.get_dummies(X_imbalanced, columns=['TLD'])
# y_imbalanced = df['label']

# print("features shape:", X_imbalanced.shape)
# print("label shape:", y_imbalanced.shape)

# X_train_imbalanced, X_test_imbalanced, y_train_imbalanced, y_test_imbalanced = train_test_split(
#     X_imbalanced, 
#     y_imbalanced, 
#     test_size=0.3, 
#     random_state=66
# )

# print("\ntraining set before resampling:")
# print(y_train_imbalanced.value_counts())
# print("\ntest set:")
# print(y_test_imbalanced.value_counts())

# from imblearn.combine import SMOTETomek
# smote_tomek = SMOTETomek(random_state=66)
# X_train_balanced, y_train_balanced = smote_tomek.fit_resample(X_train_imbalanced, y_train_imbalanced)

# print("\ntraining set  after resampling:")
# print(pd.Series(y_train_balanced).value_counts())

In [344]:
class SMOTETomekBalancer(BaseEstimator, TransformerMixin):
    def __init__(self, label_column='label', random_state=42, sampling_strategy=0.2):
        self.label_column = label_column
        self.random_state = random_state
        self.sampling_strategy = sampling_strategy
        self.smotetomek = SMOTETomek(
            random_state=self.random_state, 
            sampling_strategy=self.sampling_strategy
        )

    def fit(self, X, y=None):
        if self.label_column not in X.columns:
            raise ValueError(f"Label column '{self.label_column}' not found in input data")
            
        # Store the column names
        self.feature_names_ = [col for col in X.columns if col != self.label_column]
        self.is_fitted_ = True
        return self

    def transform(self, X):
        # Check if fit has been called
        check_is_fitted(self, 'is_fitted_')
        
        if self.label_column not in X.columns:
            raise ValueError(f"Label column '{self.label_column}' not found in input data")

        X_features = X.drop(columns=[self.label_column])
        y_label = X[self.label_column]
        
        X_resampled, y_resampled = self.smotetomek.fit_resample(X_features, y_label)
        X_resampled_df = pd.DataFrame(X_resampled, columns=self.feature_names_)
        y_resampled_df = pd.Series(y_resampled, name=self.label_column)
        
        return pd.concat([X_resampled_df, y_resampled_df], axis=1)

### IV. Normalization

In [345]:
train_set.describe()

,id,URLLength,IsDomainIP,CharContinuationRate,TLDLegitimateProb,URLCharProb,TLDLength,IsHTTPS,LineOfCode,LargestLineLength,...,Pay,Crypto,NoOfImage,NoOfJS,NoOfSelfRef,NoOfEmptyRef,label,UIQualityScore,ContentComplexity,RefPatternScore
count,93265.000000,93265.000000,93265.000000,93265.000000,93265.000000,93265.000000,93265.000000,93265.000000,93265.000000,93265.000000,...,93265.000000,93265.000000,93265.000000,93265.000000,93265.000000,93265.000000,93265.000000,93265.000000,93265.000000,93265.000000
mean,117571.859712,32.190243,0.000129,0.153689,0.372595,0.060117,2.748523,0.761765,1014.106884,2844.640755,...,0.238825,0.022592,25.340674,12.900413,74.951836,1.023374,0.936096,4.055101,53.491172,0.924458
std,68010.311816,9.533806,0.011342,0.049191,0.227803,0.004335,0.456598,0.426006,234.278134,1150.897055,...,0.426368,0.148598,15.026299,4.394515,47.838685,1.741567,0.244583,0.971644,7.710822,0.235264
min,1.000000,15.000000,0.000000,0.028986,0.000000,0.036048,2.000000,0.000000,2.000000,24.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,3.536297,0.000000
25%,58992.000000,25.000000,0.000000,0.117647,0.079963,0.058653,2.000000,1.000000,1000.000000,2586.000000,...,0.000000,0.000000,17.000000,11.000000,49.000000,0.000000,1.000000,4.000000,53.073854,0.971831
50%,116968.000000,29.000000,0.000000,0.157895,0.522907,0.060261,3.000000,1.000000,1014.000000,2681.000000,...,0.000000,0.000000,23.000000,13.000000,69.000000,0.000000,1.000000,4.000000,54.649553,1.000000
75%,176607.000000,37.000000,0.000000,0.190476,0.522907,0.062018,3.000000,1.000000,1042.000000,2909.000000,...,0.000000,0.000000,31.000000,14.000000,94.000000,1.000000,1.000000,5.000000,56.648618,1.000000
max,235795.000000,70.000000,1.000000,0.300000,0.522907,0.074789,4.000000,1.000000,1306.500000,6528.500000,...,1.000000,1.000000,54.000000,19.500000,166.000000,5.000000,1.000000,5.000000,60.809097,1.000000


In [346]:
val_set.describe()

,id,URLLength,IsDomainIP,CharContinuationRate,TLDLegitimateProb,URLCharProb,TLDLength,IsHTTPS,LineOfCode,LargestLineLength,...,Pay,Crypto,NoOfImage,NoOfJS,NoOfSelfRef,NoOfEmptyRef,label,UIQualityScore,ContentComplexity,RefPatternScore
count,39106.000000,39106.000000,39106.000000,39106.000000,39106.000000,39106.000000,39106.000000,39106.000000,39106.000000,39106.000000,...,39106.000000,39106.000000,39106.000000,39106.000000,39106.000000,39106.000000,39106.000000,39106.000000,39106.000000,39106.000000
mean,117919.766813,30.262415,0.000230,0.145012,0.373281,0.060120,2.753516,0.971769,1006.769984,2870.148187,...,0.243390,0.021531,25.055708,12.408134,74.922748,1.025469,0.928374,4.045824,53.292540,0.920960
std,68362.072087,6.481374,0.015169,0.058725,0.227418,0.004536,0.457003,0.165634,234.082167,1259.784412,...,0.429134,0.145149,14.709742,4.488498,48.133001,1.745974,0.257871,0.982260,8.155994,0.241581
min,17.000000,16.000000,0.000000,0.020000,0.000000,0.031911,2.000000,0.000000,2.000000,23.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,3.491449,0.000000
25%,57924.000000,25.000000,0.000000,0.111111,0.079963,0.058917,2.000000,1.000000,995.250000,2562.000000,...,0.000000,0.000000,17.000000,11.000000,49.000000,0.000000,1.000000,4.000000,52.748227,0.970760
50%,118242.000000,30.000000,0.000000,0.153846,0.522907,0.060276,3.000000,1.000000,1013.000000,2683.000000,...,0.000000,0.000000,23.000000,12.000000,69.000000,0.000000,1.000000,4.000000,54.646931,1.000000
75%,176874.000000,37.000000,0.000000,0.185185,0.522907,0.062072,3.000000,1.000000,1018.000000,2989.750000,...,0.000000,0.000000,31.000000,14.000000,94.000000,1.000000,1.000000,5.000000,56.510776,1.000000
max,235794.000000,63.000000,1.000000,0.320000,0.522907,0.077294,4.000000,1.000000,1283.000000,6520.000000,...,1.000000,1.000000,52.500000,20.000000,166.000000,5.000000,1.000000,5.000000,60.791422,1.000000


# 3. Compile Preprocessing Pipeline

All of the preprocessing classes or functions defined earlier will be compiled in this step.

If you use sklearn to create preprocessing classes, you can list your preprocessing classes in the Pipeline object sequentially, and then fit and transform your data.

In [347]:
pipe = Pipeline([
    ("frequency_encoder", FrequencyEncoder(columns=["TLD"], drop_original=True)),
    ("scaler_minmax", NumericMinMaxScaler()),
    ("balancer", SMOTETomekBalancer(label_column='label'))
])

train_set_preprocessed = pipe.fit_transform(train_set)
val_set_preprocessed = pipe.transform(val_set)
    


In [348]:
train_set_preprocessed.describe()

,id,URLLength,IsDomainIP,CharContinuationRate,TLDLegitimateProb,URLCharProb,TLDLength,IsHTTPS,LineOfCode,LargestLineLength,...,Crypto,NoOfImage,NoOfJS,NoOfSelfRef,NoOfEmptyRef,UIQualityScore,ContentComplexity,RefPatternScore,TLD_encoded,label
count,94506.000000,94506.000000,94506.000000,94506.000000,94506.000000,94506.000000,94506.000000,94506.000000,94506.000000,94506.000000,...,94506.000000,94506.000000,94506.000000,94506.000000,94506.000000,94506.000000,94506.000000,94506.000000,94506.000000,94506.000000
mean,117592.222705,0.317608,0.000252,0.453254,0.708584,0.612400,0.376082,0.733120,0.748870,0.423349,...,0.021489,0.449740,0.636608,0.431516,0.191969,0.788757,0.844411,0.890646,0.385092,0.869522
std,68007.943246,0.173376,0.014119,0.181791,0.431666,0.121437,0.226581,0.435771,0.216067,0.184293,...,0.144783,0.283952,0.247037,0.292973,0.341449,0.212083,0.179143,0.278243,0.459769,0.336831
min,1.000000,0.036364,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,59096.500000,0.181818,0.000000,0.297695,0.152920,0.567999,0.074359,0.031507,0.656190,0.310708,...,0.000000,0.277778,0.512821,0.241179,0.000000,0.600000,0.840639,0.965278,0.000000,1.000000
50%,117006.500000,0.261925,0.000000,0.460716,1.000000,0.625000,0.500000,1.000000,0.775776,0.408486,...,0.000000,0.425926,0.666667,0.415663,0.000000,0.800000,0.892453,1.000000,0.058250,1.000000
75%,176515.750000,0.418182,0.000000,0.584893,1.000000,0.661681,0.500000,1.000000,0.775776,0.408794,...,0.000000,0.537037,0.666667,0.536145,0.200000,1.000000,0.927357,1.000000,1.000000,1.000000
max,235795.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [349]:
val_set_preprocessed.describe()


,id,URLLength,IsDomainIP,CharContinuationRate,TLDLegitimateProb,URLCharProb,TLDLength,IsHTTPS,LineOfCode,LargestLineLength,...,Crypto,NoOfImage,NoOfJS,NoOfSelfRef,NoOfEmptyRef,UIQualityScore,ContentComplexity,RefPatternScore,TLD_encoded,label
count,37508.000000,37508.000000,37508.000000,37508.000000,37508.000000,37508.000000,37508.000000,37508.000000,37508.000000,37508.000000,...,37508.000000,37508.000000,37508.000000,37508.000000,37508.000000,37508.000000,37508.000000,37508.000000,37508.000000,37508.000000
mean,117960.411272,0.282578,0.000367,0.422929,0.709227,0.613706,0.377066,0.953778,0.753626,0.431314,...,0.021193,0.451808,0.621627,0.439563,0.197398,0.794956,0.852158,0.900384,0.383583,0.887171
std,68374.754547,0.122073,0.018211,0.216101,0.432174,0.126284,0.228004,0.197534,0.200889,0.196280,...,0.143628,0.275767,0.240989,0.292587,0.344989,0.207079,0.167411,0.266010,0.460168,0.316388
min,17.000000,0.018182,0.000000,-0.024488,0.000000,-0.106764,0.000000,0.000000,0.000000,-0.000154,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-0.000783,0.000000,0.000000,0.000000
25%,57776.500000,0.181818,0.000000,0.261228,0.152920,0.576632,0.048331,1.000000,0.681487,0.328388,...,0.000000,0.296296,0.512821,0.259036,0.000000,0.600000,0.841722,0.969072,0.000000,1.000000
50%,118663.000000,0.272727,0.000000,0.460716,1.000000,0.625379,0.500000,1.000000,0.775010,0.408794,...,0.000000,0.425926,0.615385,0.415663,0.000000,0.800000,0.892407,1.000000,0.052515,1.000000
75%,176883.250000,0.400000,0.000000,0.563928,1.000000,0.665523,0.500000,1.000000,0.775010,0.432931,...,0.000000,0.537037,0.717949,0.542169,0.200000,1.000000,0.924950,1.000000,1.000000,1.000000
max,235794.000000,0.872727,1.000000,1.073797,1.000000,1.064664,1.000000,1.000000,0.981985,0.998693,...,1.000000,0.972222,1.000000,1.000000,1.000000,1.000000,0.999691,1.000000,1.000000,1.000000


or create your own here

In [350]:
val_set_preprocessed.describe()


,id,URLLength,IsDomainIP,CharContinuationRate,TLDLegitimateProb,URLCharProb,TLDLength,IsHTTPS,LineOfCode,LargestLineLength,...,Crypto,NoOfImage,NoOfJS,NoOfSelfRef,NoOfEmptyRef,UIQualityScore,ContentComplexity,RefPatternScore,TLD_encoded,label
count,37508.000000,37508.000000,37508.000000,37508.000000,37508.000000,37508.000000,37508.000000,37508.000000,37508.000000,37508.000000,...,37508.000000,37508.000000,37508.000000,37508.000000,37508.000000,37508.000000,37508.000000,37508.000000,37508.000000,37508.000000
mean,117960.411272,0.282578,0.000367,0.422929,0.709227,0.613706,0.377066,0.953778,0.753626,0.431314,...,0.021193,0.451808,0.621627,0.439563,0.197398,0.794956,0.852158,0.900384,0.383583,0.887171
std,68374.754547,0.122073,0.018211,0.216101,0.432174,0.126284,0.228004,0.197534,0.200889,0.196280,...,0.143628,0.275767,0.240989,0.292587,0.344989,0.207079,0.167411,0.266010,0.460168,0.316388
min,17.000000,0.018182,0.000000,-0.024488,0.000000,-0.106764,0.000000,0.000000,0.000000,-0.000154,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-0.000783,0.000000,0.000000,0.000000
25%,57776.500000,0.181818,0.000000,0.261228,0.152920,0.576632,0.048331,1.000000,0.681487,0.328388,...,0.000000,0.296296,0.512821,0.259036,0.000000,0.600000,0.841722,0.969072,0.000000,1.000000
50%,118663.000000,0.272727,0.000000,0.460716,1.000000,0.625379,0.500000,1.000000,0.775010,0.408794,...,0.000000,0.425926,0.615385,0.415663,0.000000,0.800000,0.892407,1.000000,0.052515,1.000000
75%,176883.250000,0.400000,0.000000,0.563928,1.000000,0.665523,0.500000,1.000000,0.775010,0.432931,...,0.000000,0.537037,0.717949,0.542169,0.200000,1.000000,0.924950,1.000000,1.000000,1.000000
max,235794.000000,0.872727,1.000000,1.073797,1.000000,1.064664,1.000000,1.000000,0.981985,0.998693,...,1.000000,0.972222,1.000000,1.000000,1.000000,1.000000,0.999691,1.000000,1.000000,1.000000


# Cleaning & Preprocessing test_df

## Cleaning

### Handling Missing Value

In [351]:
# Assign url dengan modus ketika tidak ada Domain dan IsHTTPS
if 'URL' in test_df.columns:
    # Mencari url paling sering/modus
    most_frequent_url = test_df['URL'].mode()[0]

    mask_both_missing = pd.isna(test_df['IsHTTPS']) & pd.isna(test_df['Domain'])
    test_df.loc[mask_both_missing & pd.isna(test_df['URL']), 'URL'] = most_frequent_url

    mask_one_exists = ((pd.notna(test_df['IsHTTPS']) & pd.isna(test_df['Domain'])) | 
            (pd.isna(test_df['IsHTTPS']) & pd.notna(test_df['Domain'])))
    test_df.loc[mask_one_exists & pd.isna(test_df['URL']), 'URL'] = most_frequent_url

# Assign Domain dan TLD jika terdapat URL
for index, row in test_df.iterrows():
    if pd.notna(row['URL']):
        try:
            # Cek HTTPS
            test_df.at[index, 'IsHTTPS'] = 1 if row['URL'].startswith('https://') else 0
            
            parsed = urlparse(row['URL'])
            if not parsed.netloc: # Buat domain yang tidak punya protokol
                parsed = urlparse("http://" + row['URL'])
            
            domain = parsed.netloc or parsed.path

            # Extract TLD
            if '.' in domain:
                tld = domain.split('.')[-1]
                
                # Update Domain and TLD 
                if pd.isna(test_df.at[index, 'Domain']):
                    test_df.at[index, 'Domain'] = domain
                if pd.isna(test_df.at[index, 'TLD']):
                    test_df.at[index, 'TLD'] = tld
        except:
            continue
    # Jika URL tidak ada tapi domaion ada
    elif pd.notna(row['Domain']) and pd.isna(row['TLD']):
        domain = row['Domain']
        if '.' in domain:
            test_df.at[index, 'TLD'] = domain.split('.')[-1]
    
    # Isi URL jika Domain dan IsHTTPS ada
    if pd.isna(row['URL']) and pd.notna(row['Domain']) and pd.notna(row['IsHTTPS']):
        protocol = 'https://' if row['IsHTTPS'] == 1 else 'http://'
        test_df.at[index, 'URL'] = f"{protocol}www.{row['Domain']}"

# Proses URL-based features
if 'URL' in test_df.columns:
    # Update URLLength berdasarkan URL yang ada
    mask = test_df['URL'].notna()
    test_df.loc[mask, 'URLLength'] = test_df.loc[mask, 'URL'].str.len()
    
    # Update letter dan digit counts berdasarkan URL yang ada
    test_df.loc[mask, 'NoOfLettersInURL'] = test_df.loc[mask, 'URL'].str.count('[a-zA-Z]')
    test_df.loc[mask, 'NoOfDegitsInURL'] = test_df.loc[mask, 'URL'].str.count('[0-9]')
    test_df.loc[mask, 'NoOfEqualsInURL'] = test_df.loc[mask, 'URL'].str.count('=')
    test_df.loc[mask, 'NoOfQMarkInURL'] = test_df.loc[mask, 'URL'].str.count(r'\?')
    test_df.loc[mask, 'NoOfAmpersandInURL'] = test_df.loc[mask, 'URL'].str.count('&')
    
    url_lengths = test_df.loc[mask, 'URL'].str.len()
    test_df.loc[mask, 'LetterRatioInURL'] = test_df.loc[mask, 'NoOfLettersInURL'] / url_lengths
    test_df.loc[mask, 'DegitRatioInURL'] = test_df.loc[mask, 'NoOfDegitsInURL'] / url_lengths
    
    special_chars_pattern = '[^a-zA-Z0-9=?&]'
    test_df.loc[mask, 'NoOfOtherSpecialCharsInURL'] = test_df.loc[mask, 'URL'].str.count(special_chars_pattern)
    test_df.loc[mask, 'SpacialCharRatioInURL'] = test_df.loc[mask, 'NoOfOtherSpecialCharsInURL'] / url_lengths
    
    # CharContinuationRate
    char_series = test_df.loc[mask, 'URL']
    continuation_rates = []
    
    for url in char_series:
        if len(url) < 2:
            continuation_rates.append(0)
        else:
            continuous = sum(1 for i in range(len(url)-1) if url[i] == url[i+1])
            continuation_rates.append(continuous / (len(url)-1))
    
    test_df.loc[mask, 'CharContinuationRate'] = continuation_rates

# Proses Domain-based features
if 'Domain' in test_df.columns:
    # Update DomainLength berdasarkan Domain yang ada
    mask = test_df['Domain'].notna()
    test_df.loc[mask, 'DomainLength'] = test_df.loc[mask, 'Domain'].str.len()
    
    # Update IsDomainIP berdasarkan Domain yang ada
    test_df.loc[mask, 'IsDomainIP'] = test_df.loc[mask, 'Domain'].apply(
        lambda x: float(bool(re.match(r'^(?:[0-9]{1,3}\.){3}[0-9]{1,3}$', str(x)))))
    
    # Update NoOfSubDomain berdasarkan Domain yang ada
    test_df.loc[mask, 'NoOfSubDomain'] = test_df.loc[mask, 'Domain'].str.count(r'\.')

# Proses TLD-based features
if 'TLD' in test_df.columns:
    mask = test_df['TLD'].notna()
    test_df.loc[mask, 'TLDLength'] = test_df.loc[mask, 'TLD'].str.len()

# Hubungan HasObfuscation dengan NoOfObfuscatedChar
mask = test_df['NoOfObfuscatedChar'] == 0
test_df.loc[mask & test_df['HasObfuscation'].isna(), 'HasObfuscation'] = 0

mask = test_df['NoOfObfuscatedChar'] > 0
test_df.loc[mask & test_df['HasObfuscation'].isna(), 'HasObfuscation'] = 1

# NoOfSelfRedirect tidak boleh lebih besar dari NoOfURLRedirect
mask = test_df['NoOfURLRedirect'].notna() & test_df['NoOfSelfRedirect'].notna()
invalid_mask = mask & (test_df['NoOfSelfRedirect'] > test_df['NoOfURLRedirect'])
test_df.loc[invalid_mask, 'NoOfSelfRedirect'] = test_df.loc[invalid_mask, 'NoOfURLRedirect']

# HasExternalFormSubmit berdasarkan NoOfExternalRef
mask = test_df['NoOfExternalRef'] > 0
test_df.loc[mask & test_df['HasExternalFormSubmit'].isna(), 'HasExternalFormSubmit'] = 1

mask = test_df['NoOfExternalRef'] == 0
test_df.loc[mask & test_df['HasExternalFormSubmit'].isna(), 'HasExternalFormSubmit'] = 0

# Proses title 
mask = test_df['Title'].notna()
test_df.loc[mask & test_df['HasTitle'].isna(), 'HasTitle'] = 1
test_df.loc[~mask & test_df['HasTitle'].isna(), 'HasTitle'] = 0

# Isi missing values yang tersisa
boolean_cols = ['IsDomainIP', 'HasObfuscation', 'IsHTTPS', 'HasTitle',
                'HasFavicon', 'IsResponsive', 'HasDescription',
                'HasExternalFormSubmit', 'HasSocialNet', 'HasSubmitButton',
                'HasHiddenFields', 'HasPasswordField', 'HasCopyrightInfo']

count_cols = ['NoOfSubDomain', 'NoOfObfuscatedChar', 'NoOfLettersInURL',
              'NoOfDegitsInURL', 'NoOfEqualsInURL', 'NoOfQMarkInURL', 
              'NoOfAmpersandInURL', 'NoOfOtherSpecialCharsInURL',
              'NoOfURLRedirect', 'NoOfSelfRedirect', 'NoOfPopup',
              'NoOfiFrame', 'NoOfImage', 'NoOfCSS', 'NoOfJS',
              'NoOfSelfRef', 'NoOfEmptyRef', 'NoOfExternalRef']

# Handle missing values sisanya
for col in boolean_cols:
    if col in test_df.columns and test_df[col].isnull().any():
        test_df[col] = test_df[col].fillna(test_df[col].mode()[0])

for col in count_cols:
    if col in test_df.columns and test_df[col].isnull().any():
        test_df[col] = test_df[col].fillna(test_df[col].median())

# Handle remaining numerical columns
numerical_cols = test_df.select_dtypes(include=['float64', 'int64']).columns
remaining_cols = [col for col in numerical_cols if col not in boolean_cols + count_cols]
for col in remaining_cols:
    if test_df[col].isnull().any():
        test_df[col] = test_df[col].fillna(test_df[col].median())

print("\nJumlah missing values setelah diisi:")
print(test_df.isnull().sum())


Jumlah missing values setelah diisi:
id                               0
URL                              0
URLLength                        0
Domain                        2185
DomainLength                     0
IsDomainIP                       0
TLD                           3476
CharContinuationRate             0
TLDLegitimateProb                0
URLCharProb                      0
TLDLength                        0
NoOfSubDomain                    0
HasObfuscation                   0
NoOfObfuscatedChar               0
ObfuscationRatio                 0
NoOfLettersInURL                 0
LetterRatioInURL                 0
NoOfDegitsInURL                  0
DegitRatioInURL                  0
NoOfEqualsInURL                  0
NoOfQMarkInURL                   0
NoOfAmpersandInURL               0
NoOfOtherSpecialCharsInURL       0
SpacialCharRatioInURL            0
IsHTTPS                          0
LineOfCode                       0
LargestLineLength                0
HasTitle         

In [352]:
test_df

,id,URL,URLLength,Domain,DomainLength,IsDomainIP,TLD,CharContinuationRate,TLDLegitimateProb,URLCharProb,...,Bank,Pay,Crypto,HasCopyrightInfo,NoOfImage,NoOfCSS,NoOfJS,NoOfSelfRef,NoOfEmptyRef,NoOfExternalRef
0,48,https://www.iaee.org,20.0,NaN,12.0,0.0,org,0.263158,0.079963,0.081395,...,1.0,0.0,0.0,1.0,4.0,1.0,13.0,194.0,0.0,65.0
1,68,http://www.iran-edi.com,23.0,NaN,16.0,0.0,NaN,0.181818,0.079963,0.069183,...,0.0,0.0,0.0,0.0,0.0,1.0,2.0,0.0,0.0,1.0
2,76,https://www.bistum-chur.ch,26.0,www.bistum-chur.ch,18.0,0.0,ch,0.160000,0.004983,0.050344,...,0.0,0.0,0.0,1.0,5.0,12.0,18.0,193.0,0.0,196.0
3,155,https://www.numberthreebath.com,31.0,NaN,23.0,0.0,com,0.166667,0.522907,0.059065,...,0.0,0.0,0.0,0.0,4.0,1.0,10.0,12.0,0.0,11.0
4,167,https://ipfs.litnet.work/ipfs/bafybeib5jvxytzb...,100.0,ipfs.litnet.work,16.0,0.0,work,0.030303,0.079963,0.035754,...,0.0,0.0,0.0,0.0,4.0,1.0,3.0,2.0,0.0,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,235694,https://756-107694hr4g.weeblysite.com/,38.0,756-107694hr4g.weeblysite.com,29.0,0.0,com,0.081081,0.522907,0.057273,...,0.0,0.0,0.0,0.0,0.0,0.0,3.0,2.0,0.0,4.0
9996,235718,https://www.www.interactive-sonification.org,44.0,www.interactive-sonification.org,32.0,0.0,org,0.139535,0.079963,0.066188,...,0.0,0.0,0.0,0.0,1.0,1.0,5.0,23.0,0.0,9.0
9997,235733,https://fuchsiabrownsysadmin--ficohsa121.repl....,72.0,NaN,20.0,0.0,NaN,0.042254,0.005977,0.052057,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0
9998,235740,https://www.benoit-mccarthy.com,31.0,www.benoit-mccarthy.com,23.0,0.0,NaN,0.166667,0.522907,0.063081,...,0.0,0.0,0.0,1.0,11.0,22.0,3.0,45.0,0.0,4.0


In [353]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 54 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   id                          10000 non-null  int64  
 1   URL                         10000 non-null  object 
 2   URLLength                   10000 non-null  float64
 3   Domain                      7815 non-null   object 
 4   DomainLength                10000 non-null  float64
 5   IsDomainIP                  10000 non-null  float64
 6   TLD                         6524 non-null   object 
 7   CharContinuationRate        10000 non-null  float64
 8   TLDLegitimateProb           10000 non-null  float64
 9   URLCharProb                 10000 non-null  float64
 10  TLDLength                   10000 non-null  float64
 11  NoOfSubDomain               10000 non-null  float64
 12  HasObfuscation              10000 non-null  float64
 13  NoOfObfuscatedChar          1000

### Feature Engineering

In [354]:
# Buat fitur baru dari karakteristik URL
if 'URL' in test_df.columns:
   # Hitung karakter spesial
   test_df['special_char_count'] = test_df['URL'].str.count(r'[^a-zA-Z0-9]').fillna(0)
   # Hitung digit
   test_df['digit_count'] = test_df['URL'].str.count(r'[0-9]').fillna(0)

# Buat fitur interaksi
if 'URLLength' in test_df.columns and 'DomainLength' in test_df.columns:
   test_df['url_domain_ratio'] = test_df['URLLength'] / test_df['DomainLength'].replace(0, 1)

# Buat fitur binary dari TLD
if 'TLD' in test_df.columns:
   common_tlds = ['com', 'org', 'net', 'edu']
   test_df['is_common_tld'] = test_df['TLD'].isin(common_tlds).astype(int)

# Buat aggregate features
security_cols = ['HasObfuscation', 'IsHTTPS', 'HasHiddenFields']
test_df['security_score'] = test_df[security_cols].sum(axis=1)

# Cek fitur baru
print("\nJumlah kolom setelah feature engineering:", len(test_df.columns))
print("\nBeberapa baris dengan fitur baru:")
print(test_df[['special_char_count', 'digit_count', 'url_domain_ratio',
          'is_common_tld', 'security_score']].head())


Jumlah kolom setelah feature engineering: 59

Beberapa baris dengan fitur baru:
   special_char_count  digit_count  url_domain_ratio  is_common_tld  \
0                   5            0          1.666667              1   
1                   6            0          1.437500              0   
2                   6            0          1.444444              0   
3                   5            0          1.347826              1   
4                   9           11          6.250000              0   

   security_score  
0             1.0  
1             0.0  
2             1.0  
3             1.0  
4             2.0  


In [355]:
# Define the features to drop with reasons
features_to_drop = {
    #korelasi tinggi
    'NoOfLettersInURL': 'almost perfect correlation sama URLLength (0.998)',
    'NoOfOtherSpecialCharsInURL': 'strong correlation URLLength (0.876)',
    'DomainLength': 'strong correlation URLLength (0.845)',
    
    # varianceny 0, ga contribute ke modeling
    'NoOfObfuscatedChar': 'konstan, std 0',
    'URLTitleMatchScore': 'konstan, std 0',
    'NoOfURLRedirect': 'konstan, std 0',
    'ObfuscationRatio': 'konstan, std 0',
    'DomainTitleMatchScore': 'konstan, std 0',
    'DegitRatioInURL': 'konstan, std 0',
    'NoOfEqualsInURL': 'konstan, std 0',
    'HasObfuscation': 'konstan, std 0',
    'NoOfSelfRedirect': 'konstan, std 0',
    'NoOfPopup': 'konstan, std 0',
    'NoOfQMarkInURL': 'konstan, std 0',
    'NoOfAmpersandInURL': 'konstan, std 0',
    'NoOfDegitsInURL': 'konstan, std 0',
    'NoOfSubDomain': 'konstan, std 0',
    
    # uda kegambarin di feature lain/raw text
    'Domain': 'raw text',
    'URL': 'raw text',
    'Title': 'raw text',
    'LetterRatioInURL': 'derived',
    'SpacialCharRatioInURL': 'derived',
    
    # missing too much, low mi
    'NoOfCSS': 'High missing values, low importance',
    'HasCopyrightInfo': 'High missing values, low importance',
    'HasPasswordField': 'High missing values, low importance',
    'HasSocialNet': 'High missing values, low importance',
    'NoOfExternalRef': 'High missing, info captured in HasExternalFormSubmit'
}

#drop
test_df = test_df.drop(columns=list(features_to_drop.keys()), errors='ignore')

#same columns
common_columns = test_df.columns.intersection(val_set.columns)
test_df = test_df[common_columns]

#verify
print("\nnumber of remaining features:", len(test_df.columns))
print("training set:", test_df.columns.tolist())


# shapess
print("\nNew shapes:")
print(f"training set: {test_df.shape}")



number of remaining features: 27
training set: ['id', 'URLLength', 'IsDomainIP', 'TLD', 'CharContinuationRate', 'TLDLegitimateProb', 'URLCharProb', 'TLDLength', 'IsHTTPS', 'LineOfCode', 'LargestLineLength', 'HasTitle', 'HasFavicon', 'Robots', 'IsResponsive', 'HasDescription', 'NoOfiFrame', 'HasExternalFormSubmit', 'HasSubmitButton', 'HasHiddenFields', 'Bank', 'Pay', 'Crypto', 'NoOfImage', 'NoOfJS', 'NoOfSelfRef', 'NoOfEmptyRef']

New shapes:
training set: (10000, 27)


In [356]:
def engineer_features(dataset):
    features = dataset.copy()
    
    
    # ui content - ngambil juga dari tucil
    ui_features = ['HasFavicon',  'HasSubmitButton', 'HasTitle', 'HasDescription', 'IsResponsive']
    features['UIQualityScore'] = features[ui_features].sum(axis=1)
    
    # terkait duit n pembayaran
    financial_features = ['Bank', 'Pay', 'Crypto']
    features['FinancialRiskScore'] = (
        features[financial_features].sum(axis=1) * 
        (1 - features['IsHTTPS'])  #kalo ga https worse
    )
    
    # domain
    features['DomainTrustScore'] = (
        features['TLDLegitimateProb'] * 
        (1 + features['IsHTTPS']) *  #kalo https kasi plus
        (1 - features['IsDomainIP'])  #kalo domainnya ip = veri sus kurangin
    )
    
    # kompleksitas code
    features['ContentComplexity'] = (
        np.log1p(features['LineOfCode']) * 
        np.log1p(features['LargestLineLength'])
    )
    
    # ref pattern
    features['RefPatternScore'] = (
        features['NoOfSelfRef'] / 
        (features['NoOfSelfRef'] + features['NoOfEmptyRef']).clip(lower=1)
    )
    
    return features

test_df_engineered = engineer_features(test_df)

test_df = test_df_engineered

new_features = [
    'UIQualityScore',
    'FinancialRiskScore',
    'DomainTrustScore',
    'ContentComplexity',
    'RefPatternScore'
]


In [357]:
test_df = test_df.drop(columns = ['DomainTrustScore', 'FinancialRiskScore'])
test_df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 30 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   id                     10000 non-null  int64  
 1   URLLength              10000 non-null  float64
 2   IsDomainIP             10000 non-null  float64
 3   TLD                    6524 non-null   object 
 4   CharContinuationRate   10000 non-null  float64
 5   TLDLegitimateProb      10000 non-null  float64
 6   URLCharProb            10000 non-null  float64
 7   TLDLength              10000 non-null  float64
 8   IsHTTPS                10000 non-null  float64
 9   LineOfCode             10000 non-null  float64
 10  LargestLineLength      10000 non-null  float64
 11  HasTitle               10000 non-null  float64
 12  HasFavicon             10000 non-null  float64
 13  Robots                 10000 non-null  float64
 14  IsResponsive           10000 non-null  float64
 15  Has

# 4. Modeling and Validation

Modelling is the process of building your own machine learning models to solve specific problems, or in this assignment context, predicting the target feature `label`. Validation is the process of evaluating your trained model using the validation set or cross-validation method and providing some metrics that can help you decide what to do in the next iteration of development.

## A. KNN

### From Scratch

In [358]:
class KNNClassifier(BaseEstimator):
    def __init__(self, k=3, metric='euclidean', p=2):
        self.k = k
        self.metric = metric
        self.p = p

    def fit(self, X, y):
        self.X_train = X.to_numpy()
        self.y_train = y.to_numpy()
        return self

    def _euclidean_distance(self, x1, x2):
        return np.sqrt(np.sum((x1 - x2) ** 2))

    def _manhattan_distance(self, x1, x2):
        return np.sum(np.abs(x1 - x2))

    def _minkowski_distance(self, x1, x2, p):
        return np.power(np.sum(np.power(np.abs(x1 - x2), p)), 1 / p)

    def _calculate_distances(self, x_test):
        distances = []
        for x_train in self.X_train:
            if self.metric == 'euclidean':
                dist = self._euclidean_distance(x_test, x_train)
            elif self.metric == 'manhattan':
                dist = self._manhattan_distance(x_test, x_train)
            elif self.metric == 'minkowski':
                dist = self._minkowski_distance(x_test, x_train, self.p)
            else:
                raise ValueError("Invalid metric. Use 'euclidean', 'manhattan', or 'minkowski'.")
            distances.append(dist)
        return np.array(distances)

    def _get_neighbors(self, x_test):
        distances = self._calculate_distances(x_test)
        k_indices = np.argsort(distances)[:self.k]
        return self.y_train[k_indices]

    def _get_majority_vote(self, labels):
        labels = list(labels)
        unique_labels = list(set(labels))
        votes = [labels.count(label) for label in unique_labels]
        return unique_labels[np.argmax(votes)]

    def predict(self, X):
        X_test = X.to_numpy()
        predictions = []
        for x in X_test:
            neighbors = self._get_neighbors(x)
            majority_label = self._get_majority_vote(neighbors)
            predictions.append(majority_label)
        return np.array(predictions)

### From Scikit-Learn

In [359]:
knn_scikit = KNeighborsClassifier(n_neighbors=5)

## B. Naive Bayes

### From Scratch

In [360]:
class GaussianNaiveBayes(BaseEstimator):
    def fit(self, X, y):
        self.classes = np.unique(y)
        self.mean_std = {}
        self.prior = {}

        for c in self.classes:
            X_c = X[y == c]
            self.mean_std[c] = {
                'mean': np.mean(X_c, axis=0),
                'std': np.where(np.std(X_c, axis=0) == 0, 1e-6, np.std(X_c, axis=0))
            }
            self.prior[c] = len(X_c) / len(y)
        return self

    def _gaussian_probability(self, x, mean, std):
        epsilon = 1e-9  # Batas probabilitas minimum
        exponent = np.exp(-((x - mean) ** 2) / (2 * std ** 2))
        prob = exponent / (np.sqrt(2 * np.pi) * std)
        return np.maximum(prob, epsilon)  # Pastikan probabilitas tidak nol

    def _predict_sample(self, x):
        probabilities = {}
        for c in self.classes:
            prior = np.log(self.prior[c])
            mean = self.mean_std[c]['mean']
            std = self.mean_std[c]['std']
            likelihood = np.sum(np.log(self._gaussian_probability(x, mean, std)))
            probabilities[c] = prior + likelihood
        return max(probabilities, key=probabilities.get)

    def predict(self, X):
        return np.array([self._predict_sample(x) for x in X])


### From Scikit-Learn

In [361]:
nb_scikit = GaussianNB()

## Train and Evaluate

In [362]:
# Step 2: Apply fit_transform to the entire training set and transform val set
train_preprocessed = pipe.fit_transform(train_set)
val_preprocessed = pipe.transform(val_set)

X_train = train_preprocessed.drop(columns=['label', 'id'])
y_train = train_preprocessed['label']
X_val = val_preprocessed.drop(columns=['label', 'id'])
y_val = val_preprocessed['label']

### KNN

In [363]:
# X_train_small, X_unused, y_train_small, y_unused = train_test_split(
#     X_train, y_train, test_size=0.9, random_state=42
# )

# knn = KNNClassifier(k=5, metric='euclidean')
# knn.fit(X_train_small, y_train_small)
# knn_scratch_predictions = knn.predict(X_val)

In [364]:
# # Save model
# pickle.dump(knn, open('knn_from_scratch_model.pkl', 'wb'))
# print("KNN From Scratch model saved!")

# # Load model
# loaded_knn = pickle.load(open('knn_from_scratch_model.pkl', 'rb'))
# print("KNN From Scratch model loaded!")

In [365]:
# print(classification_report(y_val, knn_scratch_predictions))

# loaded_knn_from_scratch_predictions = loaded_knn.predict(X_val)
# print(classification_report(y_val, loaded_knn_from_scratch_predictions))

In [366]:
knn_scikit.fit(X_train, y_train)
knn_scikit_predictions = knn_scikit.predict(X_val)



In [367]:
# Save the KNN model
pickle.dump(knn_scikit, open('knn_scikit_model.pkl', 'wb'))
print("KNN Scikit-Learn model saved!")

# Load the KNN model
loaded_knn_scikit = pickle.load(open('knn_scikit_model.pkl', 'rb'))
print("KNN Scikit-Learn model loaded!")

KNN Scikit-Learn model saved!
KNN Scikit-Learn model loaded!


In [368]:
print(classification_report(y_val, knn_scikit_predictions))

loaded_knn_scikit_predictions = loaded_knn_scikit.predict(X_val)
print(classification_report(y_val, loaded_knn_scikit_predictions))

              precision    recall  f1-score   support

           0       0.97      0.84      0.90      4232
           1       0.98      1.00      0.99     33276

    accuracy                           0.98     37508
   macro avg       0.98      0.92      0.95     37508
weighted avg       0.98      0.98      0.98     37508

              precision    recall  f1-score   support

           0       0.97      0.84      0.90      4232
           1       0.98      1.00      0.99     33276

    accuracy                           0.98     37508
   macro avg       0.98      0.92      0.95     37508
weighted avg       0.98      0.98      0.98     37508



### Naive Bayes

In [369]:
nb = GaussianNaiveBayes()
nb.fit(X_train.values, y_train.values)
nb_scratch_predictions = nb.predict(X_val.values)



In [370]:
# Save the Naive Bayes model
pickle.dump(nb, open('nb_from_scratch_model.pkl', 'wb'))
print("Naive Bayes From Scratch model saved!")

# Load the Naive Bayes model
loaded_nb = pickle.load(open('nb_from_scratch_model.pkl', 'rb'))
print("Naive Bayes From Scratch model loaded!")

Naive Bayes From Scratch model saved!
Naive Bayes From Scratch model loaded!


In [371]:
print(classification_report(y_val, nb_scratch_predictions))

loaded_nb_from_scratch_predictions = loaded_nb.predict(X_val.values)
print(classification_report(y_val, loaded_nb_from_scratch_predictions))

              precision    recall  f1-score   support

           0       0.73      0.97      0.83      4232
           1       1.00      0.95      0.97     33276

    accuracy                           0.96     37508
   macro avg       0.86      0.96      0.90     37508
weighted avg       0.97      0.96      0.96     37508

              precision    recall  f1-score   support

           0       0.73      0.97      0.83      4232
           1       1.00      0.95      0.97     33276

    accuracy                           0.96     37508
   macro avg       0.86      0.96      0.90     37508
weighted avg       0.97      0.96      0.96     37508



In [372]:
nb_scikit.fit(X_train, y_train)
nb_scikit_predictions = nb_scikit.predict(X_val)



In [373]:
# Save the Naive Bayes model
pickle.dump(nb_scikit, open('nb_scikit_model.pkl', 'wb'))
print("Naive Bayes Scikit-Learn model saved!")

# Load the Naive Bayes model
loaded_nb_scikit = pickle.load(open('nb_scikit_model.pkl', 'rb'))
print("Naive Bayes Scikit-Learn model loaded!")

Naive Bayes Scikit-Learn model saved!
Naive Bayes Scikit-Learn model loaded!


In [374]:
print(classification_report(y_val, nb_scikit_predictions))

loaded_nb_scikit_predictions = loaded_nb_scikit.predict(X_val)
print(classification_report(y_val, loaded_nb_scikit_predictions))

              precision    recall  f1-score   support

           0       0.73      0.97      0.83      4232
           1       1.00      0.95      0.97     33276

    accuracy                           0.96     37508
   macro avg       0.86      0.96      0.90     37508
weighted avg       0.97      0.96      0.96     37508

              precision    recall  f1-score   support

           0       0.73      0.97      0.83      4232
           1       1.00      0.95      0.97     33276

    accuracy                           0.96     37508
   macro avg       0.86      0.96      0.90     37508
weighted avg       0.97      0.96      0.96     37508



## C. Improvements (Optional)

- **Visualize the model evaluation result**

This will help you to understand the details more clearly about your model's performance. From the visualization, you can see clearly if your model is leaning towards a class than the others. (Hint: confusion matrix, ROC-AUC curve, etc.)

- **Explore the hyperparameters of your models**

Each models have their own hyperparameters. And each of the hyperparameter have different effects on the model behaviour. You can optimize the model performance by finding the good set of hyperparameters through a process called **hyperparameter tuning**. (Hint: Grid search, random search, bayesian optimization)

- **Cross-validation**

Cross-validation is a critical technique in machine learning and data science for evaluating and validating the performance of predictive models. It provides a more **robust** and **reliable** evaluation method compared to a hold-out (single train-test set) validation. Though, it requires more time and computing power because of how cross-validation works. (Hint: k-fold cross-validation, stratified k-fold cross-validation, etc.)

## D. Submission
To predict the test set target feature and submit the results to the kaggle competition platform, do the following:
1. Create a new pipeline instance identical to the first in Data Preprocessing
2. With the pipeline, apply `fit_transform` to the original training set before splitting, then only apply `transform` to the test set.
3. Retrain the model on the preprocessed training set
4. Predict the test set
5. Make sure the submission contains the `id` and `label` column.

Note: Adjust step 1 and 2 to your implementation of the preprocessing step if you don't use pipeline API from `sklearn`.

In [375]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 30 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   id                     10000 non-null  int64  
 1   URLLength              10000 non-null  float64
 2   IsDomainIP             10000 non-null  float64
 3   TLD                    6524 non-null   object 
 4   CharContinuationRate   10000 non-null  float64
 5   TLDLegitimateProb      10000 non-null  float64
 6   URLCharProb            10000 non-null  float64
 7   TLDLength              10000 non-null  float64
 8   IsHTTPS                10000 non-null  float64
 9   LineOfCode             10000 non-null  float64
 10  LargestLineLength      10000 non-null  float64
 11  HasTitle               10000 non-null  float64
 12  HasFavicon             10000 non-null  float64
 13  Robots                 10000 non-null  float64
 14  IsResponsive           10000 non-null  float64
 15  Has

In [376]:
pipe_test = Pipeline([
    ("frequency_encoder", FrequencyEncoder(columns=["TLD"], drop_original=True)),
    ("scaler_minmax", NumericMinMaxScaler())
])

train_set_preprocessed = pipe_test.fit_transform(train_set)

test_df = pipe_test.transform(test_df)


In [377]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 30 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   id                     10000 non-null  int64  
 1   URLLength              10000 non-null  float64
 2   IsDomainIP             10000 non-null  float64
 3   CharContinuationRate   10000 non-null  float64
 4   TLDLegitimateProb      10000 non-null  float64
 5   URLCharProb            10000 non-null  float64
 6   TLDLength              10000 non-null  float64
 7   IsHTTPS                10000 non-null  float64
 8   LineOfCode             10000 non-null  float64
 9   LargestLineLength      10000 non-null  float64
 10  HasTitle               10000 non-null  float64
 11  HasFavicon             10000 non-null  float64
 12  Robots                 10000 non-null  float64
 13  IsResponsive           10000 non-null  float64
 14  HasDescription         10000 non-null  float64
 15  NoO

In [384]:
X_test = test_df.drop(columns=['id'])

nb_scratch_predictions = nb.predict(X_test.values)

In [385]:
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')

# Nama file dengan timestamp
filename = f'submission_{timestamp}.csv'

# Buat DataFrame untuk submission
submission = pd.DataFrame({
    'id': test_df['id'],
    'label': nb_scratch_predictions
})

# Simpan ke CSV dengan nama file yang dinamis
submission.to_csv(filename, index=False)

# 6. Error Analysis

Based on all the process you have done until the modeling and evaluation step, write an analysis to support each steps you have taken to solve this problem. Write the analysis using the markdown block. Some questions that may help you in writing the analysis:

- Does my model perform better in predicting one class than the other? If so, why is that?
- To each models I have tried, which performs the best and what could be the reason?
- Is it better for me to impute or drop the missing data? Why?
- Does feature scaling help improve my model performance?
- etc...

`Provide your analysis here`